# Ipynb to test different classifiers

and different ML models.


A partir de las 3 tablas ya procesdas : 
* homeantennas
* sumlinks
* groundtruth

Procedemos a utilizar los clasificadores para ver prediccion

In [1]:
## seteamos los parametros del notebook
%autosave 180
import pandas as pd; 
import numpy as np; 
import os;
import random;
import time
%matplotlib inline
import numpy as np; import os;import random;
import graphlab as gl
#esto es para dibujar directo a la notebook
gl.canvas.set_target('ipynb')

#seteamos el lugar de trabajo
HOMEDIR=os.path.expanduser('~')
DATADIR=HOMEDIR+"/mobility-study/mexico-scripts-ver2/datasets"
DATADIR2 = HOMEDIR+'/mobility-study/mexico-scripts-ver2/data'
# os.chdir(DATADIR)


Autosaving every 180 seconds


In [2]:
!ls $DATADIR

gtruth_0114_0715  gtruth_0215_0715  homeant  sl  sl.csv.gz


In [3]:
def get_input_sframe(ds):
    if ds == 'home_antenna': url = DATADIR +"/homeant"
    elif ds == 'sum_links': url  = DATADIR +"/sl"
    elif ds == 'gtruth_02': url  = DATADIR +"/gtruth_0215_0715" # can be near ground truth (previous to july 2015)
    elif ds == 'gtruth_01': url  =  DATADIR +"/gtruth_0114_0715" # or can be old GT
    else: print('type chosen is %s, type should be home_antenna, sum_links, gtruth_02 or gtruth_01' % ds)
    return url 

In [4]:
url = get_input_sframe('gtruth_01')
!du -sh $url

92M	/home/juan/mobility-study/mexico-scripts-ver2/datasets/gtruth_0114_0715


In [5]:
#from unbalanced_dataset import UnderSampler, NearMiss, CondensedNearestNeighbour, OneSidedSelection,\
#NeighbourhoodCleaningRule, TomekLinks, ClusterCentroids, OverSampler, SMOTE,\
#SMOTETomek, SMOTEENN, EasyEnsemble, BalanceCascade

In [6]:
%%time

sf_table = gl.load_sframe(get_input_sframe('home_antenna'))


sl_table = gl.load_sframe(get_input_sframe('sum_links'))

gt_table = gl.load_sframe(get_input_sframe('gtruth_01'))


rename_gt = (dict([(col,col+"_gt") for col in gt_table.column_names() if col != 'USER']))

#agrego la etiqueta "_gt" a las columnas del ground_truth
gt_table.rename(rename_gt) 

/home/juan/mfixman/venv/local/lib/python2.7/site-packages/requests/packages/urllib3/util/ssl_.py:334: SNIMissingWarning: An HTTPS request has been made, but the SNI (Subject Name Indication) extension to TLS is not available on this platform. This may cause the server to present an incorrect TLS certificate, which can cause validation failures. You can upgrade to a newer version of Python to solve this. For more information, see https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  SNIMissingWarning
/home/juan/mfixman/venv/local/lib/python2.7/site-packages/requests/packages/urllib3/util/ssl_.py:132: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. You can upgrade to a newer version of Python to solve this. For more information, see https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecurePlatformWarning


This non-commercial license of GraphLab Create for academic use is assigned to jdemonasterio@dc.uba.ar and will expire on May 06, 2018.


[INFO] graphlab.cython.cy_server: GraphLab Create v1.10.1 started. Logging: /tmp/graphlab_server_1494805233.log


CPU times: user 576 ms, sys: 140 ms, total: 716 ms
Wall time: 2.09 s


## simple format description

Aca miramos las top 10 antenas que utilizo un usuario, ordeandas de 0 a 10 segun el uso, en numero de llamados, que el user le daba a c. antenna. _0_ Seria la antenna mas utilizada y _10_ la menos utilizada. El count es la cantidad de veces que utilizo esa antenna. 

Cuando aparece _WEEKNIGHT_ en el nombre de la columna, es porque cumple la condicion de que los llamados fueron hechos de noche fuera del horario [8,20] y dentro de la semana laboral.

Siguiendo las definiciones del trabajo de Caro, un user es _EPIDEMIC_ siii su ANTENNA_WEEKNIGHT_0 (esta es la home_antenna) pertence a la zona epidemica.

El mobility_diameter es el radio de las antennas (0 si uso una sola, etc.) utilizadas por este user. Nuevamente el modificador _WEEKNIGHT_ solo aplica para antennas utilizadas en esos horarios.

In [7]:
# column names
sf_table.column_names()

['USER',
 'COUNT_0',
 'COUNT_1',
 'COUNT_2',
 'COUNT_3',
 'COUNT_4',
 'COUNT_5',
 'COUNT_6',
 'COUNT_7',
 'COUNT_8',
 'COUNT_9',
 'ANTENNA_ID_0',
 'ANTENNA_ID_1',
 'ANTENNA_ID_2',
 'ANTENNA_ID_3',
 'ANTENNA_ID_4',
 'ANTENNA_ID_5',
 'ANTENNA_ID_6',
 'ANTENNA_ID_7',
 'ANTENNA_ID_8',
 'ANTENNA_ID_9',
 'COUNT_WEEKNIGHT_0',
 'COUNT_WEEKNIGHT_1',
 'COUNT_WEEKNIGHT_2',
 'COUNT_WEEKNIGHT_3',
 'COUNT_WEEKNIGHT_4',
 'COUNT_WEEKNIGHT_5',
 'COUNT_WEEKNIGHT_6',
 'COUNT_WEEKNIGHT_7',
 'COUNT_WEEKNIGHT_8',
 'COUNT_WEEKNIGHT_9',
 'ANTENNA_ID_WEEKNIGHT_0',
 'ANTENNA_ID_WEEKNIGHT_1',
 'ANTENNA_ID_WEEKNIGHT_2',
 'ANTENNA_ID_WEEKNIGHT_3',
 'ANTENNA_ID_WEEKNIGHT_4',
 'ANTENNA_ID_WEEKNIGHT_5',
 'ANTENNA_ID_WEEKNIGHT_6',
 'ANTENNA_ID_WEEKNIGHT_7',
 'ANTENNA_ID_WEEKNIGHT_8',
 'ANTENNA_ID_WEEKNIGHT_9',
 'MOBILITY_DIAMETER',
 'MOBILITY_DIAMETER_WEEKNIGHT']

## sum links description

La tabla sum_links tiene mas atributos y con data mas rica en informacion:
    
Esencialmente para c/ user miramos la cantidad de llamados y el tiempo que duraron esos llamados pero segmentando con distintos modificadores. `Time` representa la duracion del llamado y Calls el conteo de llamados.

Los modficadores/ segmentaciones son:

* mes en el cual estamos parados (12 == diciembre, ..., 08 == agosto)
* OUT/IN, separa por la direccion de los llamados si salientes u entrantes.
* _VULN_ : separa los llamados que fueron realizados hacia/desde un target_user (en una llamada hay 2 usuarios, el origin o el target) viviendo en una zona epidemica. Donde la home antena de un target_user determina su vulnerabilidad segun si es zona epidemica o no.
* Weekend, WeekDay y WeekNight son lo que suenan. Weekend el finde, Weeknight la semana pero fuera de horario laboral y Weekday en horario laboral y de lunes a viernes.

Hay solo una columna que no entra enteramente en este esquema que es VULNERABLE. Esta columna hace un conteo p/c/ usuario d cuantos target_users viven en una zona epidemica. Tambien se segmenta esta columna con los modficiadores anteriores (el mes y el out/in).

In [8]:
# column names
sl_table.column_names()

['USER',
 'TimeWeekEnd_OUT_12',
 'CallsWeekDay_OUT_12',
 'CallsWeekNight_OUT_12',
 'TimeWeekDay_OUT_12',
 'CallsWeekEnd_OUT_12',
 'TimeWeekNight_OUT_12',
 'TimeWeekEnd_OUT_VUL_12',
 'CallsWeekDay_OUT_VUL_12',
 'CallsWeekNight_OUT_VUL_12',
 'TimeWeekDay_OUT_VUL_12',
 'CallsWeekEnd_OUT_VUL_12',
 'TimeWeekNight_OUT_VUL_12',
 'VULNERABLE_OUT_12',
 'TimeWeekEnd_IN_12',
 'CallsWeekDay_IN_12',
 'CallsWeekNight_IN_12',
 'TimeWeekDay_IN_12',
 'CallsWeekEnd_IN_12',
 'TimeWeekNight_IN_12',
 'TimeWeekEnd_IN_VUL_12',
 'CallsWeekDay_IN_VUL_12',
 'CallsWeekNight_IN_VUL_12',
 'TimeWeekDay_IN_VUL_12',
 'CallsWeekEnd_IN_VUL_12',
 'TimeWeekNight_IN_VUL_12',
 'VULNERABLE_IN_12',
 'TimeWeekEnd_OUT_08',
 'CallsWeekDay_OUT_08',
 'CallsWeekNight_OUT_08',
 'TimeWeekDay_OUT_08',
 'CallsWeekEnd_OUT_08',
 'TimeWeekNight_OUT_08',
 'TimeWeekEnd_OUT_VUL_08',
 'CallsWeekDay_OUT_VUL_08',
 'CallsWeekNight_OUT_VUL_08',
 'TimeWeekDay_OUT_VUL_08',
 'CallsWeekEnd_OUT_VUL_08',
 'TimeWeekNight_OUT_VUL_08',
 'VULNERABLE_OUT_0

# Para la
tabla gt (ground_truth) es mas simple la explicacion. Solo se busco el antenna_ID_0 (nuevamente la antenna mas utilizada) por un user

In [9]:
gt_table.head()

USER,ANTENNA_ID_0_gt,ANTENNA_ID_WEEKNIGHT_0_gt
41D91B8CB8747902A80D6108D548A8AF ...,2898,2898
24A624FC979DF41830A31C7CED65E2D2 ...,4285,4285
C3FF4C9281746CAC6C09802C254BDE71 ...,4336,4336
212D3A9745F1347063267844311B31FF ...,1200,884
544695294E816F26691C4D01516ABF9B ...,195,195
FEC4BD059A42FB5283E07539C1FD2CB1 ...,262,262
BDEF57DB3EF1035FF3F210B47FC2A5B0 ...,1557,4021
3AE1DCA333E55CA87E4F5331035DB600 ...,608,608
2A72E07E48667A6289D93874A623C152 ...,2302,2302
C5287E4004BE8F3A12AD489FE8204056 ...,2644,828


# We need antennas metadata
to get the epidemicity of each antenna and add that info to the dataset

In [10]:
#agregamos al gt su info de epidemicidad
antennas_file = DATADIR2+'/celdas_limpio.csv'
antennas = gl.SFrame.read_csv(antennas_file, 
                              delimiter= "|", 
                usecols=['LATITUDE','LONGITUDE','CEL_ID','STATE','EPIDEMIC'],
                column_type_hints=[float, float, str,str, bool])

Finished parsing file /home/juan/mobility-study/mexico-scripts-ver2/data/celdas_limpio.csv

Parsing completed. Parsed 4415 lines in 0.059146 secs.

In [11]:
# agrego tambien data de epidemicidad al simpleformat table
sf_table = sf_table.join(antennas['CEL_ID','EPIDEMIC','STATE'], on = {'ANTENNA_ID_WEEKNIGHT_0':'CEL_ID'},how = 'left')

In [12]:
%%time
# enriquecemos la data de gt con info de epidemicidad
gt_table = gt_table.join(antennas['CEL_ID','EPIDEMIC','STATE'], on = {'ANTENNA_ID_WEEKNIGHT_0_gt':'CEL_ID'},how = 'left')
gt_table.rename({'EPIDEMIC':'EPIDEMIC_gt'});

CPU times: user 21.8 s, sys: 3.15 s, total: 25 s
Wall time: 3.07 s


In [13]:
gt_table.column_names()

['USER',
 'ANTENNA_ID_0_gt',
 'ANTENNA_ID_WEEKNIGHT_0_gt',
 'EPIDEMIC_gt',
 'STATE']

In [14]:
gt_table.head()

USER,ANTENNA_ID_0_gt,ANTENNA_ID_WEEKNIGHT_0_gt,EPIDEMIC_gt,STATE
41D91B8CB8747902A80D6108D548A8AF ...,2898,2898,1,Oaxaca
24A624FC979DF41830A31C7CED65E2D2 ...,4285,4285,0,Mexico
C3FF4C9281746CAC6C09802C254BDE71 ...,4336,4336,0,Mexico
212D3A9745F1347063267844311B31FF ...,1200,884,1,Jalisco
544695294E816F26691C4D01516ABF9B ...,195,195,0,Chihuahua
FEC4BD059A42FB5283E07539C1FD2CB1 ...,262,262,0,Guanajuato
BDEF57DB3EF1035FF3F210B47FC2A5B0 ...,1557,4021,1,Tabasco
3AE1DCA333E55CA87E4F5331035DB600 ...,608,608,0,Mexico
2A72E07E48667A6289D93874A623C152 ...,2302,2302,0,Distrito_Federal
C5287E4004BE8F3A12AD489FE8204056 ...,2644,828,0,Queretaro


### Sample data in case we want to work
with a smaller data sized set to test different algorithms

we will do this on sf_table which is our leading DF. 

All other frames will be joined onto this one, thus sampling the main DF is enough to sample the final DF.

In [16]:
# sample_data = 0.2
sample_data = None

if sample_data:
    sf_table, _ = sf_table.random_split(sample_data, seed=2015)

# create data matrix

In [17]:
%%time
## integrate target to single frame

data = sf_table.join(gt_table['USER','EPIDEMIC_gt'], on = 'USER', how = 'inner')
data = data.join(sl_table, on = 'USER', how = 'inner')

#no podemos tener nulls en el target asi que dropeamos
data = data.dropna(columns = ['EPIDEMIC_gt'], how='any')
print(data.shape)
del sl_table, sf_table, gt_table

(1483010, 176)
CPU times: user 5min 30s, sys: 24.2 s, total: 5min 54s
Wall time: 2min 27s


### sub_sample rows in case we want to work
with a better balanced dataset among the two attributes `EPIDEMIC` and `EPIDEMIC_gt`.

In [20]:
%time
# 1st check group sizes

epidemic_transition = data.groupby(['EPIDEMIC','EPIDEMIC_gt'],
                             {'count':gl.aggregate.COUNT_DISTINCT('USER')})

epidemic_transition

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 10 µs


EPIDEMIC,EPIDEMIC_gt,count
1,1,384466
1,0,16642
0,1,21007
0,0,1060895


### note
some users don't have current epidemicity since they haven't used their cellphones enough on weeknights to be evaluated (only at other times). We must later exclude them from analysis

This check is here

In [21]:
col = 'EPIDEMIC'
mask = data[col] == None

res = data[mask]['EPIDEMIC','ANTENNA_ID_WEEKNIGHT_0']
print(res.shape)
res.head(3)

(0, 2)


EPIDEMIC,ANTENNA_ID_WEEKNIGHT_0


### now sub_sample big groups of users who didn't migrate i.e. which have same epidemic and epidemic_gt status

In [23]:

sample_cols = ['EPIDEMIC','EPIDEMIC_gt']
# data[sample_cols].head(5)

In [24]:
%time
sub_sample_data = 0.08 # this is the percentage of users we would like to keep from the big gruops
# sub_sample_data = None

if sub_sample_data:
    excluded_users_list = data[(data['EPIDEMIC']==  data['EPIDEMIC_gt'])]['USER'].sample(1-sub_sample_data)
    excluded_users_list = excluded_users_list.to_numpy()
    data = data[data['USER'].is_in(excluded_users_list)==0] # dirty trick to flip values of binary series
    

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 10 µs


In [25]:
# check results of sub_sampling
epidemic_transition = data.groupby(['EPIDEMIC','EPIDEMIC_gt'],
                             {'count':gl.aggregate.COUNT_DISTINCT('USER')})
epidemic_transition


EPIDEMIC,EPIDEMIC_gt,count
1,1,30702
1,0,16642
0,1,21007
0,0,84659


In [26]:
epidemic_transition

EPIDEMIC,EPIDEMIC_gt,count
1,1,30702
1,0,16642
0,1,21007
0,0,84659


In [27]:
# data.shape

## select our global model

sometimes we wish to try different models. 
For example:

* focus on migrations of currently non-vulnerable users
* exclude certain columns that are very informative or correlated with the target


In [ ]:
sub_sample_file = '/'.join(get_input_sframe('sum_links').split('/')[:-1]) + '/data_balanced_sample.csv'
sub_sample_file

data.export_csv(sub_sample_file)

In [ ]:
# res = data.shape
# res

In [ ]:
# df_test = data.to_dataframe()

In [ ]:
# df_test.head(5)

# p% of set as validation and the resulting  as train

In [45]:
split_perc = 0.7
data, val_set = data.random_split(split_perc, seed=2015)

# Instance fast model with MNB
#### convertimos la data en numpy, previamente limpiando las columnas de antennas ya que el clasificador no acepta strings

In [46]:
from sklearn.naive_bayes import *
from sklearn.metrics import *
from sklearn.cross_validation import *
from sklearn.grid_search import *

/home/juan/mobility-study/scikit-learn/sklearn/cross_validation.py:43: DeprecationWarning: This module has been deprecated in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/juan/mobility-study/scikit-learn/sklearn/grid_search.py:43: DeprecationWarning: This module has been deprecated in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [47]:
## preparamos los datasets que no pueden tomar valores negativos o categorical vars

## drop certain columns for different experiments

we are going to try different models

In [63]:
exclude_cols = [     
     'EPIDEMIC',
                ]

In [ ]:
data = data[[col for col in data.column_names() if col not in exclude_cols]]

In [48]:
%%time
X = data[[col for col in data.column_names() if col != 'USER' and col != 'EPIDEMIC_gt' \
          and not('ANTENNA' in col) ]]

Y = data['EPIDEMIC_gt']

X_val = val_set[[col for col in data.column_names() if col != 'USER' and \
                 col != 'EPIDEMIC_gt' and not('ANTENNA' in col )]]
Y_val = val_set['EPIDEMIC_gt']

for col in [col for col in X.column_names() if 'COUNT' in col]:
    X[col]= X[col].apply(lambda x :  x if x>=0 else 0)
    X_val[col]= X_val[col].apply(lambda x :  x if x>=0 else 0)

    

CPU times: user 12.4 s, sys: 616 ms, total: 13 s
Wall time: 5.66 s


In [62]:
# X.column_names()

In [32]:
%%time
X = X.to_numpy()
Y = Y.to_numpy()
X_val = X_val.to_numpy()
Y_val = Y_val.to_numpy()

CPU times: user 3min 9s, sys: 15.7 s, total: 3min 25s
Wall time: 2min 52s


## remember no negative values

In [57]:
%%time
start_time = time.time()
param_grid = {'alpha':[1e-3, 1e-1, 1e1,1e0], 'fit_prior': [True,False],
             }

mnb  = MultinomialNB( )

clf = GridSearchCV(mnb, param_grid, scoring='f1_weighted', fit_params=None, n_jobs=-1, iid=True, refit=True, 
verbose=3, pre_dispatch='2*n_jobs', error_score='raise')

clf2 = MultinomialNB( )
#how many parameters to randomly search for
#n_iter_search = 45

#random_search = RandomizedSearchCV(rforest, param_distributions=param_grid,
                                 #  n_iter=n_iter_search, n_jobs =8, verbose=3)

elapsed_time =  time.time() - start_time

#Y = categorical(train_table_target.values, drop=True).astype(int)

clf.fit(X,Y)

elapsed_time =   time.time() - start_time 

print('Grid Search took %s seconds to run' % (elapsed_time))

cv_result = pd.DataFrame(columns=['mean_score' ] + [key for key, value in clf.grid_scores_[0].parameters.items() ] )
i=0
for setup in clf.grid_scores_:
    row = [clf.grid_scores_[i].mean_validation_score ] + [value for key, value in clf.grid_scores_[i].parameters.items() ]
    cv_result.loc[i] = row
    i+=1


#este flag es basicamente para chequear que hayamos terminado nomas
finished = True

print('\n Best estimator was %s \n' % str(clf.best_estimator_))
print('\n Best estimator was %s \n' % str(clf.best_score_))

clf2.set_params(**clf.best_params_)

clf2.fit(X,Y)


#converted_dict = evaluation_print(clf, X_val, test_table_target.values, test_table.index.values, 
#                              test_table_target[test_table_target>0].index.values,start_date,future)

print(classification_report(Y_val,clf2.predict(X_val)))

Fitting 3 folds for each of 8 candidates, totalling 24 fits
[CV] alpha=0.001, fit_prior=True .....................................
[CV] ............ alpha=0.001, fit_prior=True, score=0.927250 -   4.0s
[CV] alpha=0.001, fit_prior=True .....................................
[CV] ............ alpha=0.001, fit_prior=True, score=0.976829 -   4.0s
[CV] alpha=0.001, fit_prior=True .....................................
[CV] ............ alpha=0.001, fit_prior=True, score=0.918735 -   5.1s
[CV] alpha=0.001, fit_prior=False ....................................
[CV] ........... alpha=0.001, fit_prior=False, score=0.927126 -   5.9s
[CV] alpha=0.001, fit_prior=False ....................................
[CV] ........... alpha=0.001, fit_prior=False, score=0.976823 -   4.0s
[CV] alpha=0.001, fit_prior=False ....................................
[CV] ........... alpha=0.001, fit_prior=False, score=0.918739 -   4.0s
[CV] alpha=0.1, fit_prior=True .......................................
[CV] ............

[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed:  1.9min finished


Grid Search took 115.099725008 seconds to run

 Best estimator was MultinomialNB(alpha=0.001, class_prior=None, fit_prior=True) 


 Best estimator was 0.940938151577 

             precision    recall  f1-score   support

          0       0.96      0.96      0.96    323278
          1       0.89      0.90      0.90    122186

avg / total       0.94      0.94      0.94    445464

CPU times: user 4min 22s, sys: 53.6 s, total: 5min 15s
Wall time: 3min 20s


# agregamos la data de roc auc

In [58]:
roc_auc_score(Y_val,clf2.predict(X_val))

0.93145104767684839

In [59]:
coln = 2
print(cv_result.groupby(cv_result.columns[coln])['mean_score'].mean())
print(cv_result.groupby(cv_result.columns[coln])['mean_score'].std())

fit_prior
False    0.940896
True     0.940938
Name: mean_score, dtype: float64
fit_prior
False    0.0
True     0.0
Name: mean_score, dtype: float64


In [42]:
from sklearn.ensemble import RandomForestClassifier
from graphlab import random_forest_classifier, boosted_trees_classifier
from graphlab.toolkits import cross_validation, model_parameter_search

In [17]:
param_grid = {'criterion': ['gini'], 'n_estimators': [15,30,50],
  'max_features': ["auto", "log2"], "bootstrap": [ False, True],
    "min_samples_leaf": np.append(np.random.randint(3,15,3),[3]),'max_depth':[10,20,30], 
               "class_weight": ['balanced']
              }


# armamos con Gr Boost un clasificador pero ahora el target es ver los movimientos 0 a 0 , 0 a 1, 1a 0 , 1 a 1

In [20]:
a = data['EPIDEMIC','EPIDEMIC_gt'].head(3)
a[0] == [0,0]

False

In [21]:
#son (00)=0, (01)=1, 10 =2, 11 =3 
def get_new_target(row):
    rv=-1
    if row['EPIDEMIC'] ==0 and row['EPIDEMIC_gt'] == 0: rv=0
    if row['EPIDEMIC'] ==0 and row['EPIDEMIC_gt'] == 1: rv=1
    if row['EPIDEMIC'] ==1 and row['EPIDEMIC_gt'] == 0: rv=2
    if row['EPIDEMIC'] ==1 and row['EPIDEMIC_gt'] == 1: rv=3
    return rv

In [52]:
%%time
data['Y'] =  data['EPIDEMIC','EPIDEMIC_gt'].apply(lambda row: get_new_target(row))
val_set['Y'] =  val_set['EPIDEMIC','EPIDEMIC_gt'].apply(lambda row: get_new_target(row))

CPU times: user 1min 2s, sys: 3.84 s, total: 1min 6s
Wall time: 17.7 s


# get kfolds

In [25]:
%%time
num_folds = 4
kfold = cross_validation.KFold(data[[col for col in data.column_names() \
                                    if not('EPIDEMIC' in col) and col !='USER']],num_folds=num_folds)

CPU times: user 27.3 s, sys: 1.4 s, total: 28.7 s
Wall time: 27.6 s


# Boosted Classifier
### HyperParams search

In [136]:
# Boosted Trees HyperParams
params = {
    'target': 'Y', ## the target column string name 
    'max_iterations': [2,6,9,12], #The maximum number of iterations for boosting. Each iteration == extra tree.
    #'class_weights': [None, 'auto'], #Weights the examples in the training data according to the given class weights.
    'class_weights': [None],
    'max_depth': [2,6,9,12], #Maximum depth of a tree. Must be at least 1.
    'step_size': [1e-1,0.5,1,1.5], # Step size (shrinkage) used in update to prevents overfitting
    'min_loss_reduction': [1e-2,1e-1,1,10], #Minimum loss reduction required to make a further partition/split a node during the tree learning
    'min_child_weight': [1e-2,1e-1,1,10], # Controls the minimum weight of each leaf node . larger values > less overfitting
    'row_subsample': [0.01,0.1,0.5], #Subsample the ratio of the training set in each iteration of tree construction
    'column_subsample': [0.01,0.1,0.5], # Subsample the ratio of the columns in each iteration of tree construction
    #'metric': ['accuracy', 'auc', 'f1_score','recall','precision'], # Performance metric(s) that are tracked during training     
    'metric': ['f1_score'],
    'random_seed' : int(abs(hash('im not joking...'))%1e6) 
}

model_factory = boosted_trees_classifier.create

### Random Search Boosted 
# Full Throtttttttleee

In [137]:
%%time
start_time = time.time()
model_search = model_parameter_search.create(datasets = datasets = kfold, 
                                             #[data, val_set],
                                             model_factory = model_factory,
            model_parameters = params, perform_trial_run = True  )

search_results = model_search.get_results()

all_time = start_time - time.time()

2016-07-17 15:30:09,900 [INFO] graphlab.deploy.job, 22: Validating job.
2016-07-17 15:30:09,908 [INFO] graphlab.deploy.map_job, 186: Validation complete. Job: 'Model-Parameter-Search-Jul-17-2016-15-30-0900000' ready for execution
2016-07-17 15:30:15,037 [INFO] graphlab.deploy.map_job, 192: Job: 'Model-Parameter-Search-Jul-17-2016-15-30-0900000' scheduled.
2016-07-17 15:31:27,791 [INFO] graphlab.deploy.job, 22: Validating job.
2016-07-17 15:31:27,795 [INFO] graphlab.deploy.map_job, 220: A job with name 'Model-Parameter-Search-Jul-17-2016-15-30-0900000' already exists. Renaming the job to 'Model-Parameter-Search-Jul-17-2016-15-30-0900000-ed86b'.
2016-07-17 15:31:27,804 [INFO] graphlab.deploy.map_job, 186: Validation complete. Job: 'Model-Parameter-Search-Jul-17-2016-15-30-0900000-ed86b' ready for execution
2016-07-17 15:31:32,276 [INFO] graphlab.deploy.map_job, 192: Job: 'Model-Parameter-Search-Jul-17-2016-15-30-0900000-ed86b' scheduled.
2016-07-17 15:31:35,289 [INFO] graphlab.deploy.job

In [138]:
search_results.shape

(8, 15)

In [140]:
search_results.head()

Columns:
	class_weights	float
	column_subsample	float
	max_depth	int
	max_iterations	int
	metric	str
	min_child_weight	float
	min_loss_reduction	float
	random_seed	int
	row_subsample	float
	step_size	float
	target	str
	num_folds	int
	fold_id	list
	mean_training_accuracy	float
	model_id	list

Rows: 8

Data:
+---------------+------------------+-----------+----------------+----------+
| class_weights | column_subsample | max_depth | max_iterations |  metric  |
+---------------+------------------+-----------+----------------+----------+
|      None     |       0.1        |     12    |       6        | f1_score |
|      None     |       0.1        |     6     |       12       | f1_score |
|      None     |       0.5        |     6     |       9        | f1_score |
|      None     |       0.1        |     12    |       9        | f1_score |
|      None     |       0.01       |     2     |       2        | f1_score |
|      None     |       0.1        |     2     |       9        | f1_score |
|      None     |       0.01       |     9     |       9        | f1_score |
|      None     |       0.5        |     12    |       6        | f1_score |
+---------------+------------------+-----------+----------------+----------+
+------------------+--------------------+-------------+---------------+-----------+
| min_child_weight | min_loss_reduction | random_seed | row_subsample | step_size |
+------------------+--------------------+-------------+---------------+-----------+
|       10.0       |        0.1         |    415168   |      0.5      |    0.5    |
|       10.0       |        0.1         |    415168   |      0.1      |    1.0    |
|       10.0       |        10.0        |    415168   |      0.01     |    0.1    |
|       10.0       |        10.0        |    415168   |      0.5      |    1.0    |
|       0.1        |        0.1         |    415168   |      0.01     |    1.0    |
|       1.0        |        10.0        |    415168   |      0.01     |    1.0    |
|       0.01       |        0.01        |    415168   |      0.01     |    0.5    |
|       0.01       |        0.01        |    415168   |      0.01     |    0.5    |
+------------------+--------------------+-------------+---------------+-----------+
+--------+-----------+---------+------------------------+----------+
| target | num_folds | fold_id | mean_training_accuracy | model_id |
+--------+-----------+---------+------------------------+----------+
|   Y    |     1     |   [0]   |     0.94124886993      |   [14]   |
|   Y    |     1     |   [0]   |     0.961610376793     |   [12]   |
|   Y    |     1     |   [0]   |     0.976458875076     |   [18]   |
|   Y    |     1     |   [0]   |     0.983330859547     |   [16]   |
|   Y    |     1     |   [0]   |     0.721053331611     |   [0]    |
|   Y    |     1     |   [0]   |     0.961723142878     |   [6]    |
|   Y    |     1     |   [0]   |     0.913090118414     |   [4]    |
|   Y    |     1     |   [0]   |     0.994986246393     |   [10]   |
+--------+-----------+---------+------------------------+----------+
[8 rows x 15 columns]

In [143]:
for col in [col for col in search_results.column_names() if \
            not('accuracy' in col) and col!= 'model_id' and col!= 'random_seed' and col!= 'fold_id' \
                               and col!= 'num_folds']:
    print(search_results.groupby(col,
                             {'mean':gl.aggregate.MEAN('mean_training_accuracy'),
                                'std':gl.aggregate.STD('mean_training_accuracy')}))

+---------------+-----------------+---------------+
| class_weights |       std       |      mean     |
+---------------+-----------------+---------------+
|      None     | 0.0831262575802 | 0.93168772758 |
+---------------+-----------------+---------------+
[1 rows x 3 columns]

+------------------+------------------+----------------+
| column_subsample |       std        |      mean      |
+------------------+------------------+----------------+
|       0.5        | 0.00926368565827 | 0.985722560735 |
|       0.01       | 0.0960183934014  | 0.817071725013 |
|       0.1        | 0.0148815451391  | 0.961978312287 |
+------------------+------------------+----------------+
[3 rows x 3 columns]

+-----------+------------------+----------------+
| max_depth |       std        |      mean      |
+-----------+------------------+----------------+
|     2     |  0.120334905633  | 0.841388237244 |
|     6     | 0.00742424914172 | 0.969034625935 |
|     12    | 0.0230806497161  | 0.973188658623

## hyperparameter results

* class_weights: None, gana por afano
* column_subsample : algo menor a 1 pero no queda claro si cuanto menos mejor
* row_subsample : algo menor a 1 pero no queda claro si cuanto menos mejor, refinar
* max_depth: varia mucho, refinar
* max_iterations: tampoco queda claro si poquito o mucho es mejor, reprobar
* metric: el f1_score es el unico que no parece variar mucho,los demas cambian demasiado
* min_child_weight: varia mucho, reprobar con mas
* min_loss_reduction: varia mucho, reprobar
* step_size: mas grande parece ser mejor> 0.1

# Gl Random Forest HyperParam Tuning

# OJO ACA EL TARGET CAMBIO!

In [39]:
data.column_names()

['USER',
 'TimeWeekEnd_OUT_12',
 'CallsWeekDay_OUT_12',
 'CallsWeekNight_OUT_12',
 'TimeWeekDay_OUT_12',
 'CallsWeekEnd_OUT_12',
 'TimeWeekNight_OUT_12',
 'TimeWeekEnd_OUT_VUL_12',
 'CallsWeekDay_OUT_VUL_12',
 'CallsWeekNight_OUT_VUL_12',
 'TimeWeekDay_OUT_VUL_12',
 'CallsWeekEnd_OUT_VUL_12',
 'TimeWeekNight_OUT_VUL_12',
 'VULNERABLE_OUT_12',
 'TimeWeekEnd_IN_12',
 'CallsWeekDay_IN_12',
 'CallsWeekNight_IN_12',
 'TimeWeekDay_IN_12',
 'CallsWeekEnd_IN_12',
 'TimeWeekNight_IN_12',
 'TimeWeekEnd_IN_VUL_12',
 'CallsWeekDay_IN_VUL_12',
 'CallsWeekNight_IN_VUL_12',
 'TimeWeekDay_IN_VUL_12',
 'CallsWeekEnd_IN_VUL_12',
 'TimeWeekNight_IN_VUL_12',
 'VULNERABLE_IN_12',
 'TimeWeekEnd_OUT_08',
 'CallsWeekDay_OUT_08',
 'CallsWeekNight_OUT_08',
 'TimeWeekDay_OUT_08',
 'CallsWeekEnd_OUT_08',
 'TimeWeekNight_OUT_08',
 'TimeWeekEnd_OUT_VUL_08',
 'CallsWeekDay_OUT_VUL_08',
 'CallsWeekNight_OUT_VUL_08',
 'TimeWeekDay_OUT_VUL_08',
 'CallsWeekEnd_OUT_VUL_08',
 'TimeWeekNight_OUT_VUL_08',
 'VULNERABLE_OUT_0

In [40]:
%%time
data['Y'] =  data['EPIDEMIC_gt']
val_set['Y'] =  val_set['EPIDEMIC_gt']

CPU times: user 2min 35s, sys: 34.3 s, total: 3min 9s
Wall time: 22.1 s


In [43]:
%%time
num_folds = 4
kfold = cross_validation.KFold(data[[col for col in data.column_names() if col != 'USER' and not( 'EPIDEMIC' in col ) \
          and not('ANTENNA' in col) ]]
,num_folds=num_folds)

CPU times: user 16 ms, sys: 0 ns, total: 16 ms
Wall time: 13.4 ms


In [44]:
# Random Forest HyperParams
params = {
    'target': 'Y', ## the target column string name 
    #'max_iterations': [2,6,9,12], #The maximum number of iterations for boosting. Each iteration == extra tree.
    'class_weights': [None, 'auto'], #Weights the examples in the training data according to the given class weights.
    #'class_weights': [None],
    'max_depth': [2,6,9,12], #Maximum depth of a tree. Must be at least 1.
    'min_loss_reduction': [1e-2,1e-1,1,10], #Minimum loss reduction required to make a further partition/split a node during the tree learning
    'min_child_weight': [1e-2,1e-1,1,10], # Controls the minimum weight of each leaf node . larger values > less overfitting
    'row_subsample': [0.01,0.1,0.5], #Subsample the ratio of the training set in each iteration of tree construction
    'column_subsample': [0.01,0.1,0.5], # Subsample the ratio of the columns in each iteration of tree construction
    #'metric': ['accuracy', 'auc', 'f1_score','recall','precision'], # Performance metric(s) that are tracked during training     
    'metric': ['f1_score'],
    'random_seed' : int(abs(hash('im not joking...'))%1e6) 
}

model_factory = random_forest_classifier.create


In [ ]:
%%time
start_time = time.time()
model_search = model_parameter_search.create(datasets = kfold, 
                                             #[data, val_set],
                                             model_factory = model_factory,
            model_parameters = params, perform_trial_run = True  )

search_results = model_search.get_results()
all_time =  time.time() -start_time
print('fitting took %s seconds' % str(all_time))

[INFO] graphlab.deploy.job: Validating job.
[INFO] graphlab.deploy.job: Creating a LocalAsync environment called 'async'.
[INFO] graphlab.deploy.map_job: Validation complete. Job: 'Model-Parameter-Search-Aug-21-2016-16-08-1400000' ready for execution
[INFO] graphlab.deploy.map_job: Job: 'Model-Parameter-Search-Aug-21-2016-16-08-1400000' scheduled.


In [ ]:
print('fitting took %s seconds' % str(all_time))

In [ ]:
model_search.get_status()

In [ ]:
model_search.summary()

In [ ]:
search_results

In [ ]:
#model_search.get_metrics()['exception_message']

In [ ]:
for col in [col for col in search_results.column_names() if \
            not('accuracy' in col) and col!= 'model_id' and col!= 'random_seed' and col!= 'fold_id' \
                               and col!= 'num_folds']:
    print(search_results.groupby(col,
                             {'mean':gl.aggregate.MEAN('mean_validation_accuracy'),
                                'std':gl.aggregate.STD('mean_validation_accuracy')}))

## Refit best model for feature importance

In [ ]:
params = model_search.get_best_params()


In [42]:
%%time

best_model =  random_forest_classifier.create(dataset=data[[col for col in data.column_names() \
                                    if not('EPIDEMIC' in col) and col !='USER']],
                                             validation_set = 'auto',
                                             **params)

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.



WARNING: Ignore unsupported tracking metric f1_score

Random forest classifier:

--------------------------------------------------------

Number of examples          : 985423

...

| 97        | 1319.462815  |

| 98        | 1333.963521  |

| 99        | 1347.875469  |

| 100       | 1362.430618  |

+-----------+--------------+

In [44]:
best_feat = best_model.get_feature_importance()
best_feat

name,index,count
COUNT_0,None,536
MOBILITY_DIAMETER,None,495
MOBILITY_DIAMETER_WEEKNIGHT ...,None,479
CallsWeekDay_IN_08,None,288
COUNT_WEEKNIGHT_0,None,283
CallsWeekDay_OUT_08,None,236
COUNT_1,None,222
CallsWeekDay_OUT_12,None,217
CallsWeekDay_OUT_VUL_09,None,211
CallsWeekDay_IN_VUL_08,None,207


In [47]:
best_model.training_time

1362.4314

In [54]:
%%time
test_error = best_model.evaluate(val_set[[col for col in data.column_names() \
                                    if not('EPIDEMIC' in col) and col !='USER']])

CPU times: user 2min 2s, sys: 1.51 s, total: 2min 4s
Wall time: 24.2 s


In [55]:
test_error.

{'accuracy': 0.9317004292153799,
 'auc': 0.8479283597887244,
 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 8
 
 Data:
 +--------------+-----------------+--------+
 | target_label | predicted_label | count  |
 +--------------+-----------------+--------+
 |      2       |        3        |  2145  |
 |      1       |        3        |  2697  |
 |      1       |        0        |  3664  |
 |      3       |        3        | 103670 |
 |      3       |        0        | 12155  |
 |      2       |        0        |  2752  |
 |      0       |        3        |  7012  |
 |      0       |        0        | 311369 |
 +--------------+-----------------+--------+
 [8 rows x 3 columns],
 'f1_score': 0.464190333713697,
 'log_loss': 1.2443176674986582,
 'precision': 0.9205516478410865,
 'recall': 0.46825831776309357,
 'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 	class	int
 
 Rows: 400004
 
 Data:
 +-----------+-----+----

In [ ]:
roc_auc_score(Y_val,clf2.predict(X_val))

### Results

* bootstrap  ???= False es 5% mejor
* min_samples_leaf ????= mas chico es claramente mejor, pero tmb aumenta el overfitting lo cual me hace caer mucho el valor del recall en el test_set. Sin embargo es un parametro muy sensible en la precision. Resta evaluar asi el tradeoff entre la precision y el volumen de users al cual queremos llegar.
* n_estimators ???= mas pareceria mejor, pero depende del app y hay que ver 'cuanto' mejora por app
* citerion ???= entropy o gini no cambia. gini podria ser mejor entonces pues entropy usa logs de los valores lo cual es mas computacionalmente costoso
* max_features ???= no afecta al score. con auto esta bien
* max_depth ??=  mas es mejor. intentaria probar con >15

# full graphlab models

In [29]:
%%time
# Create the model
model = gl.classifier.create(data, target = 'EPIDEMIC_gt',
                              features = [col for col in data.column_names() if col != 'EPIDEMIC_gt' and col!= 'USER'])

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.

PROGRESS: The following methods are available for this type of problem.
PROGRESS: LogisticClassifier, SVMClassifier
PROGRESS: The returned model will be chosen according to validation accuracy.


Logistic regression:

--------------------------------------------------------

Number of examples          : 985960

Number of classes           : 2

...

| 10        | 17       | 1.000000  | 42.004133    | 0.589961          | 0.580371            |

+-----------+----------+-----------+--------------+-------------------+---------------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

PROGRESS: Model selection based on validation accuracy:
PROGRESS: ---------------------------------------------
PROGRESS: LogisticClassifier              : 0.97509
PROGRESS: SVMClassifier                   : 0.580371
PROGRESS: ---------------------------------------------
PROGRESS: Selecting LogisticClassifier based on validation set performance.


# Validamos

In [32]:
model.get_current_options

<bound method LogisticClassifier.get_current_options of Class                         : LogisticClassifier

Schema
------
Number of coefficients        : 82760
Number of examples            : 985960
Number of classes             : 2
Number of feature columns     : 173
Number of unpacked features   : 173

Hyperparameters
---------------
L1 penalty                    : 0
L2 penalty                    : 0.01

Training Summary
----------------
Solver                        : lbfgs
Solver iterations             : 2
Solver status                 : TERMINATED: Terminated due to numerical difficulties.
Training time (sec)           : 15.7768

Settings
--------
Log-likelihood                : inf

Highest Positive Coefficients
-----------------------------
ANTENNA_ID_WEEKNIGHT_7[1081]  : 89.8181
ANTENNA_ID_WEEKNIGHT_9[3195]  : 89.8181
ANTENNA_ID_WEEKNIGHT_8[1116]  : 88.2713
ANTENNA_ID_WEEKNIGHT_1[99]    : 87.7119
ANTENNA_ID_WEEKNIGHT_1[244]   : 87.1922

Lowest Negative Coefficients
------------

In [25]:
%%time
classification = model.classify(val_set)
results = model.evaluate(val_set)
results

CPU times: user 2min 24s, sys: 15.4 s, total: 2min 39s
Wall time: 18.7 s


In [28]:
model.get_current_options

<bound method LogisticClassifier.get_current_options of Class                         : LogisticClassifier

Schema
------
Number of coefficients        : 1068253
Number of examples            : 985503
Number of classes             : 2
Number of feature columns     : 174
Number of unpacked features   : 174

Hyperparameters
---------------
L1 penalty                    : 0
L2 penalty                    : 0.01

Training Summary
----------------
Solver                        : lbfgs
Solver iterations             : 10
Solver status                 : TERMINATED: Iteration limit reached.
Training time (sec)           : 45.6194

Settings
--------
Log-likelihood                : 279.9326

Highest Positive Coefficients
-----------------------------
USER[AC571EA92E8F687BE672AF55FAC1EAD3]: 162.6371
USER[6EFBFA6AC99A85C7DA609A6F9D6014F6]: 143.8373
USER[2F5E433CAEBB38733143D0A8EB5E4EB2]: 66.9896
USER[2321510F75A59BD796D0DBD1CBC13D9B]: 66.571
USER[D9AC206F61422B357DBEBC95AC894CE2]: 55.8497

Lowest Ne

# En particular queremos ver que tan bien nos va con los users

que eran == 0 en el hoy

y eran == 1 en el pasado


In [59]:
special_val = val_set[(val_set['EPIDEMIC_gt'] == 1) & (val_set['EPIDEMIC'] == 0)]
print('Cantidad de usuarios que migraron son %s' %special_val.shape[0])
results = model.evaluate(special_val)
results

Cantidad de usuarios que migraron son 6361


{'accuracy': 0.3280930671278101, 'auc': 0.0, 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 2
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      1       |        1        |  2087 |
 |      1       |        0        |  4274 |
 +--------------+-----------------+-------+
 [2 rows x 3 columns], 'f1_score': 0.4940814393939394, 'log_loss': 3.952783379671028, 'precision': 1.0, 'recall': 0.3280930671278101, 'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 100001
 
 Data:
 +-----------+-----+----------------+------+---+
 | threshold | fpr |      tpr       |  p   | n |
 +-----------+-----+----------------+------+---+
 |    0.0    | nan |      1.0       | 6361 | 0 |
 |   1e-05   | nan | 0.970444898601 | 6361 | 0 |
 |   2e-05   | nan | 0.957868259708 | 6361 | 0 |
 |   3e-05   | nan | 0.946234868731 | 6361 | 0 |
 |   4e

# por las dudas chequeo las correlaciones de la matriz a ver si algun feature "ES" el ground_truth

In [35]:
%%time
# paso a pandas
X = val_set.to_dataframe()
corr_matrix = X.corr()

CPU times: user 1min 18s, sys: 1.66 s, total: 1min 20s
Wall time: 1min 20s


In [36]:
# chequeamos correlacion especificamente con el target/gt
corr_cut = 0.5
corr_matrix[(corr_matrix['EPIDEMIC_gt']>corr_cut) | corr_matrix['EPIDEMIC_gt']<-corr_cut]['EPIDEMIC_gt']

Series([], Name: EPIDEMIC_gt, dtype: float64)

In [37]:
corr_matrix.shape

(154, 154)

# notar que en la primera corrida
el algoritmo esta hecho tomando como GT  data desde enero 2015 a julio 2015

Luego la correlacion con el GT 'decae" a medida que los meses del train_table se alejan de esta fecha (empeora en direccion a 12 que es diciembre)


In [29]:
%%time
#lo paso a una serie
corr_series = abs(corr_matrix).unstack()
#ordeno por los valores mas altos
corr_series = corr_series.sort_values(ascending = False) 
#es obvio que la diagonal tenia correlacion ==1, asi que elimino aquellos valores que eran de la diagonal
#tambien saco los valores que estan "arriba" de la diagonal, pero haciendo la comparacion entre strings
corr_series = corr_series[(corr_series.index.get_level_values(0) != corr_series.index.get_level_values(1))\
                         & (corr_series.index.get_level_values(0) <= corr_series.index.get_level_values(1))]

CPU times: user 8 ms, sys: 4 ms, total: 12 ms
Wall time: 134 ms


In [31]:
corr_series.head(15)

COUNT_8            COUNT_9              0.981473
COUNT_7            COUNT_8              0.976586
COUNT_WEEKNIGHT_8  COUNT_WEEKNIGHT_9    0.969355
COUNT_6            COUNT_7              0.968488
COUNT_WEEKNIGHT_7  COUNT_WEEKNIGHT_8    0.968116
COUNT_WEEKNIGHT_6  COUNT_WEEKNIGHT_7    0.963660
COUNT_5            COUNT_6              0.957804
COUNT_7            COUNT_9              0.956372
COUNT_WEEKNIGHT_5  COUNT_WEEKNIGHT_6    0.955928
EPIDEMIC           EPIDEMIC_gt          0.943801
COUNT_6            COUNT_8              0.943118
COUNT_WEEKNIGHT_7  COUNT_WEEKNIGHT_9    0.942522
COUNT_WEEKNIGHT_4  COUNT_WEEKNIGHT_5    0.940706
COUNT_4            COUNT_5              0.939109
COUNT_WEEKNIGHT_6  COUNT_WEEKNIGHT_8    0.935862
dtype: float64

# A partir de aca, todo lo que esta abajo es viejo. Queda a revisar

In [21]:
%%time
# miramos ahora las columnas con poca varianza ie. que aportan poca info

var_matrix = X_train.var()
#imprimimos aquellas que esten por abajo de cierto percentil
perc = 0.5
var_matrix[var_matrix<perc]

CPU times: user 41.5 s, sys: 6.67 s, total: 48.1 s
Wall time: 48.6 s


In [22]:
# como resultado de esto decidimos dropear las columnas, por poca varianza..
# [ EXPOSED ]
X_train.drop('EXPOSED',axis=1, inplace=True)

In [23]:
%%time
#con decomposicion qr tratamos de ver si existe alguna columna que sea comb linear de las demas
linear_test = np.linalg.qr(X_train.sample(1e6))[1]
#notar que devuelve la tabla de tamanyo N`columnas x Ncolumnas

#sumo a traves de las columnas para que me de el valor absoluto sumado de c/fila
linear_test = abs(linear_test.sum(axis=1))<1e-2
#si hubiese alguan que sea linearcomb entonces tendria que aparecer que toda la fila es de ceros
for i, col in enumerate(X_train.columns):
    if linear_test[i] == True:
        print(col)

/home/juan/miniconda2/lib/python2.7/site-packages/pandas/core/generic.py:2569: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  locs = rs.choice(axis_length, size=n, replace=replace, p=weights)


CPU times: user 19 s, sys: 7.15 s, total: 26.1 s
Wall time: 9.82 s


## First Simple Classification Tests
La idea es probar tres clasificadores basicos y tener una medida de precision, recall, F1-score p/c/ estimador:

* **Clasificador 1**: con la proporción de gente que vive en la zona endémica en el "presente", asignar la gente al azar respetando esa proporción i.e. $X \sim Bernoulli(\hat{p}) s.t. p$ is the mean of endemic users in the dataset

* **Clasificador 2**: usando la información del "presente", si la persona vive en la zona endémica en el presente, predecir que va a ocurrir lo mismo en el "pasado" i.e. un estimador que devuelve el feature de endemicidad actual para predecir la endemicidad en el pasado.


* **Clasificador 3**: usar 2 features del "presente", (i) si la persona vive en la zona endémica, (ii) cual es la proporción de sus contactos que viven en la zona endémica.

# Clf 1

In [24]:
from sklearn.metrics import *
from scipy import stats

/home/juan/mobility-study/scikit-learn/sklearn/externals/joblib/_multiprocessing_helpers.py:29: UserWarning: [Errno 38] Function not implemented.  joblib will operate in serial mode
  warnings.warn('%s.  joblib will operate in serial mode' % (e,))


In [35]:
p_hat = X_train['EPIDEMIC'].mean()
#the output vector, i.e. the estimated epidemic users
y_hat = stats.bernoulli.rvs(p_hat, size=len(X_train))

In [37]:
print("A priori probability of bein epidemic is %s" %p_hat)
print(classification_report(X_train['y'], y_hat ,digits=4))

A priori probability of bein epidemic is 0.293362195887
             precision    recall  f1-score   support

          0     0.7065    0.7061    0.7063   1413371
          1     0.2931    0.2935    0.2933    586914

avg / total     0.5852    0.5850    0.5851   2000285



# Clf 2

In [42]:
X_train['EPIDEMIC'].sum(),X_train['EPIDEMIC'].shape , X_train['y'].sum()

(586808, (2000285,), 586914)

In [50]:
(X_train['y'] == X_train['EPIDEMIC']).sum()

1173263

In [44]:
X_train['EPIDEMIC'].value_counts()

False    1413477
True      586808
Name: EPIDEMIC, dtype: int64

In [48]:
table = X_train.pivot_table(['EPIDEMIC', 'y'],
                                   aggfunc=np.sum)

ValueError: No group keys passed!

In [49]:
X_train.pivot_table(values=)

In [45]:
X_train['EPIDEMIC'].dtypes

dtype('bool')

In [46]:
X_train['y'].value_counts()

0    1413371
1     586914
Name: y, dtype: int64

In [41]:
y_hat = X_train['EPIDEMIC'].values

print(classification_report(y_target, y_hat, digits=4 ))

             precision    recall  f1-score   support

      False     1.0000    1.0000    1.0000   1413477
       True     1.0000    1.0000    1.0000    586808

avg / total     1.0000    1.0000    1.0000   2000285



# Clf 3
adf

adsf

asdf

In [48]:
'number of users who are and have been epidemic is %s'% (X_train['EPIDEMIC'].values == y_target).sum(), "dataset size is %s" % len(X_train)

('number of users who are and have been epidemic is 1171065',
 'dataset size is 2000285')

## Cross, test, train splt

In [49]:
%%time
#parto efectivamente el dataset en 3 conjunots, uno para fitear todo. otro para entrenar+transform y el ultimo
#para transform +val
msk = np.random.rand(len(X_train)) < 0.3333
X_fit = X_train[msk]
X_train =  X_train[~msk]
msk2 = np.random.rand(len(X_train)) < 0.5
X_val =  X_train[msk2]
X_train =  X_train[~msk2]

CPU times: user 1.46 s, sys: 544 ms, total: 2.01 s
Wall time: 2.05 s


## Count Featurizer on Antennas
nos quedamos con las columnas de antennas y en graphlab aplicamos el algo de CountFeaturizer para cada categoria de
Antenna_ID_N


In [50]:
%%time
## Count Featurizer on Antennas
#nos quedamos con las columnas de antennas y pasamos todo a Graph Lab.
import graphlab as gl
from graphlab.toolkits.feature_engineering import *
ant_cols = [col for col in X_train.columns if "ANTENNA_ID" in col]
ant_sframe_fit = gl.SFrame(X_fit[ant_cols + ['y']])
ant_sframe_train = gl.SFrame(X_train[ant_cols + ['y']])
ant_sframe_val = gl.SFrame(X_val[ant_cols + ['y']])

A newer version of GraphLab Create (v1.10.1) is available! Your current version is v1.8.5.

You can use pip to upgrade the graphlab-create package. For more information see https://dato.com/products/create/upgrade.


This non-commercial license of GraphLab Create is assigned to svega@dm.uba.ar and will expire on April 16, 2017. For commercial licensing options, visit https://dato.com/buy/.


2016-06-14 00:17:59,281 [INFO] graphlab.cython.cy_server, 176: GraphLab Create v1.8.5 started. Logging: /tmp/graphlab_server_1465863477.log


CPU times: user 7.21 s, sys: 1.51 s, total: 8.72 s
Wall time: 7.14 s


In [51]:
%%time
countfeat = gl.feature_engineering.create(ant_sframe_fit, 
               CountFeaturizer(target='y'))

# Transform the train set. This is the dataset I will train my classifier on
transformed_ant_train = countfeat.transform(ant_sframe_train)
transformed_ant_val = countfeat.transform(ant_sframe_val)

del ant_sframe_fit,ant_sframe_train,ant_sframe_val

#por alguna razon guarda los valores de probabilidad como una lista de un unico valor
for col in [col for col in transformed_ant_train.column_names() if "prob_" in col]:
    transformed_ant_train[col] = transformed_ant_train[col].apply(lambda x: x[0]) 
    transformed_ant_val[col] = transformed_ant_val[col].apply(lambda x: x[0]) 

#me quedo solo con los valores de probabilidad.
transformed_ant_train = transformed_ant_train[[col for col in transformed_ant_train.column_names() if "prob_" in col]]
transformed_ant_val = transformed_ant_val[[col for col in transformed_ant_train.column_names() if "prob_" in col]]

transformed_ant_val = transformed_ant_val.to_dataframe()
transformed_ant_train = transformed_ant_train.to_dataframe()

CPU times: user 58.6 s, sys: 8.91 s, total: 1min 7s
Wall time: 37.3 s


In [52]:
X_val[ant_cols] = transformed_ant_val.values
X_train[ant_cols] = transformed_ant_train.values
del transformed_ant_train, transformed_ant_val

## Scikit estimator fit and cross-validation

In [53]:
from sklearn.naive_bayes import *
#from sklearn.linear_model import *
from sklearn.ensemble import *
from sklearn.svm import *
from sklearn.neural_network import *
from sklearn.cross_validation import *
from sklearn.multiclass import OneVsRestClassifier
from sklearn.utils import *
from sklearn.preprocessing import label_binarize, scale, StandardScaler
from sklearn.metrics import *
from sklearn.ensemble import *
from sklearn.tree import *
from sklearn.grid_search import *
from sklearn.pipeline import Pipeline

/home/juan/mobility-study/scikit-learn/sklearn/cross_validation.py:43: DeprecationWarning: This module has been deprecated in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/juan/mobility-study/scikit-learn/sklearn/grid_search.py:43: DeprecationWarning: This module has been deprecated in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


## Create new variables
These would be more "real" variables i.e. variables that would be "humanly" explainable

In [54]:
X_train.columns
#tenemos 38 cols

Index([u'ANTENNA_ID_0', u'COUNT_0', u'ANTENNA_ID_1', u'COUNT_1',
       u'ANTENNA_ID_2', u'COUNT_2', u'ANTENNA_ID_3', u'COUNT_3',
       u'ANTENNA_ID_4', u'COUNT_4', u'ANTENNA_ID_5', u'COUNT_5',
       u'ANTENNA_ID_6', u'COUNT_6', u'ANTENNA_ID_7', u'COUNT_7',
       u'ANTENNA_ID_8', u'COUNT_8', u'ANTENNA_ID_9', u'COUNT_9',
       u'MOBILITY_DIAMETER', u'EPIDEMIC', u'CallsWeekDaylight',
       u'CallsWeekNight', u'CallsWeekend', u'TimeWeekDaylight',
       u'TimeWeekNight', u'TimeWeekend', u'CallsWeekDaylight_EPI',
       u'CallsWeekNight_EPI', u'CallsWeekend_EPI', u'TimeWeekDaylight_EPI',
       u'TimeWeekNight_EPI', u'TimeWeekend_EPI', u'TOTAL_USERS', u'EPI_USERS',
       u'EXP_USERS', u'y'],
      dtype='object')

## Create new features
these would be "synthetic" variables

In [55]:
def process_X_files(tables, scaled=False, poly2=False,train_size = 0.8):
    #como input tiene que entrar la tabla sin splitea en train_test aun
    #tiene que suceder que haya consistencia entre las columnas de c/dataframe tambien
    
    
    calls_cols = [col for col in X_train.columns if "Calls" in col] 
    time_cols = [col for col in X_train.columns if "Time" in col]
    epi_calls_cols = [col for col in X_train.columns if "Calls" in col and "EPI" in col] 
    epi_time_cols = [col for col in X_train.columns if "Time" in col and "EPI" in col]
    antenna_cols = [col for col in X_train.columns if "ANTENNA_" in col]
    count_cols = [col for col in X_train.columns if "COUNT_" in col]
    condition_cols = [col for col in X_train.columns if "EXPOSED" == col or "EPIDEMIC" == col]
    
    categorical_cols = condition_cols
    numerical_cols = [col for col in table.columns if col not in categorical_cols]

    print("First dataframe is %s big" % str(train_table.shape))
    #print("Test dataframe is %s big" % str(test_table.shape))
    
    X_train_categorical = train_table[categorical_cols].values
    X_train_numeric = train_table[numeric_cols].values
    
    if scaled ==True :
            min_max_scaler = MinMaxScaler().fit(X_train_count_time)
            X_train_count_time = min_max_scaler.transform(X_train_numeric)
    
    #hacemos interacciones polinomiales sobre las columnas de count/time
    if poly2 == True:
        #pensar que hacer polynomial features es masomenos como agregar nˆ2 nuevas columnas
        #con lo cual tenemos que tener cuidado en no reventar la memoria, ponemor las primeras 50 columnas 
        #como tope para aplicar interacciones polinomiales

        poly2_transform = PolynomialFeatures(degree=2, interaction_only=False, include_bias=False)
        max_cols = 50

        if  (X_train_count_time.shape[1] <= max_cols):
            max_cols = X_train_count_time.shape[1]
        poly2_transform.fit(X_train_count_time[:,0:max_cols])

        X_train_poly2 = poly2_transform.transform(X_train_count_time[:,0:max_cols])
        X_train_count_time = np.concatenate( ( X_train_poly2, X_train_count_time[:,max_cols:]),axis=1) 

    X_train = np.concatenate( ( X_train_count_time, X_train_categorical),axis=1)
    
    processed_tables = [X_train]
    
    #aca basicamente replicamos lo anterior pero para todo el resto de las tablas y utilizando los fits que ya tenemos

    for i in range(len(tables)):
        # skip the X_train table which comes in the first postion and will behave as our special fitting table
        if i ==0:
            continue
        print(i)
        X_val_categorical = tables[i][categorical_cols].values
        X_val_count_time = tables[i][count_time_cols].values

        if scaled ==True :
            X_val_count_time = min_max_scaler.transform(X_val_count_time)

        #hacemos interacciones polinomiales sobre las columnas de count/time
        if poly2 == True:
            #pensar que hacer polynomial features es masomenos como agregar nˆ2 nuevas columnas
            #con lo cual tenemos que tener cuidado en no reventar la memoria, ponemor las primeras 50 columnas 
            #como tope para aplicar interacciones polinomiales

            X_val_poly2 = poly2_transform.transform(X_val_count_time[:,0:max_cols])
            X_val_count_time = np.concatenate( ( X_val_poly2, X_val_count_time[:,max_cols:]),axis=1) 

        X_val = np.concatenate( ( X_val_count_time, X_val_categorical),axis=1)
        
        processed_tables = processed_tables + [X_val]
    
    for i in range(len(processed_tables)):    
        print("Table {0} shape is {1}".format( i,str(processed_tables[i].shape)))
#        print("Table %s categorical shape is %s" % str(X_val_categorical.shape))
#        print("Val non-categorical shape is %s" % str(X_val_count_time.shape))
#        print("Val categorical shape is %s" % str(X_val_categorical.shape))
    
    return processed_tables

## GradBoost model

In [56]:
## seteo los generalizadores de CV
##genero los estimadores
W = np.array([15 if i == 1 else 1 for i in y])
#model = GradientBoostingClassifier()

gradboost = GradientBoostingClassifier( loss = 'deviance',n_estimators=20, 
                                       max_depth = 20
                                 )
svc = LinearSVC(C=2.0, tol = 1e-4,
                class_weight = 'balanced'
               )

svc2 = SVC(C=2.0, kernel = 'rbf' ,decision_function_shape='ovr',tol = 1e-5,
                class_weight = 'balanced', max_iter = 1000
               )


NameError: name 'y' is not defined

# Multinomial Naive Bayes - Benchmark - Cross Validation
mnb es bastante rapido y eficiente para rapidamente resultados en problemas de clasificacion supervisada

In [79]:
start_time = time.time()
param_grid = {'alpha':[0,1.0, 1e-1,1e1], 'fit_prior': [True,False],
             }
estimator  = MultinomialNB( )

clf =GridSearchCV(estimator, param_grid, scoring='roc_auc', fit_params=None, n_jobs=10, iid=True, refit=True, 
verbose=3, pre_dispatch='2*n_jobs', error_score='raise')
n_iter_search =60

#how many parameters to randomly search for
n_iter_search = 45

#clf = RandomizedSearchCV(estimator, param_distributions=param_grid,
#                                   n_iter=n_iter_search, n_jobs =-1, verbose=3)
X = X_train[X_train.columns[:-1]].values
Y = X_train['y'].values

#X = X_train_categorical
clf.fit(X,Y)


#random_search.fit(X_train,y_train)
elapsed_time =  time.time() - start_time

print('Random Search took %s seconds to run' % elapsed_time)


all_time =   time.time() - start_time 
print('Grid Search took %s seconds to run' % (all_time - elapsed_time))


X_test = X_val[X_train.columns[:-1]].values
Y_test = X_val['y'].values

predicted_proba = pd.DataFrame()
predicted_proba['prob'] = clf.predict_proba(X_test).max(axis=1)
predicted_proba.index = X_val.index
predicted_proba['class'] = clf.predict(X_test)
predicted_proba['gt'] =  Y_test
predicted_proba.index.name = "USER"
predicted_proba.sort_values(by = 'prob',ascending = False, inplace = True)

cut_percentages = [10,20,30,40]
#converted = (predicted_proba['class']==1)
#me fijo todos los users que van a convertir en el futuro

past_epidemic = X_val[X_val['y']==1].index.values

start = 0
mobility_dict = {}
index_segments = []
for i in cut_percentages:
    cut_size = int(len(predicted_proba)*i/100.0)    
    bool_array = (~np.ones(len(predicted_proba), dtype=bool))
    bool_array[start: start + cut_size] = True
    index_segments += [bool_array]
    start = start + cut_size
    #subtable = 
    mobility_dict[str(i)] = predicted_proba.loc[index_segments[-1]].index.values
    
    print(classification_report(predicted_proba.loc[index_segments[-1],"gt"].values,
                            predicted_proba.loc[index_segments[-1],'class'].values ))
    
    
print(classification_report(predicted_proba.loc[:,"gt"].values,
                            predicted_proba.loc[:,'class'].values ))
    
    
print("* Rates for distinct percentage groups \n")
for i in cut_percentages:
    subtable = mobility_dict[str(i)]

    print("{0} %% group rate: {1} size: {2}".format(str(i), 
                                np.in1d(subtable,past_epidemic).sum()*1.0/len(subtable),len(subtable))) 
    
    
cv_result = pd.DataFrame(columns=['mean_score' ] + [key for key, value in clf.grid_scores_[0].parameters.items() ] )
i=0
for setup in clf.grid_scores_:
    row = [clf.grid_scores_[i].mean_validation_score ] + [value for key, value in clf.grid_scores_[i].parameters.items() ]
    cv_result.loc[i] = row
    i+=1

    
    
all_time =   time.time() - start_time

#este flag es basicamente para chequear que hayamos terminado nomas
finished = True

print('This cell took %s seconds to run' % all_time)

Fitting 3 folds for each of 12 candidates, totalling 36 fits
[CV] alpha=0, fit_prior=True .........................................
[CV] ................ alpha=0, fit_prior=True, score=0.297282 -   2.9s
[CV] alpha=0, fit_prior=True .........................................
[CV] ................ alpha=0, fit_prior=True, score=0.294858 -   2.9s
[CV] alpha=0, fit_prior=True .........................................
[CV] ................ alpha=0, fit_prior=True, score=0.295935 -   2.9s
[CV] alpha=0, fit_prior=False ........................................
[CV] ............... alpha=0, fit_prior=False, score=0.297254 -   2.9s
[CV] alpha=0, fit_prior=False ........................................
[CV] ............... alpha=0, fit_prior=False, score=0.294880 -   2.9s
[CV] alpha=0, fit_prior=False ........................................
[CV] ............... alpha=0, fit_prior=False, score=0.295969 -   3.0s
[CV] alpha=0.5, fit_prior=True .......................................
[CV] ...........

[Parallel(n_jobs=10)]: Done  12 tasks      | elapsed:   45.9s


[CV] .............. alpha=1.0, fit_prior=True, score=0.297282 -   2.9s
[CV] alpha=1.0, fit_prior=True .......................................
[CV] .............. alpha=1.0, fit_prior=True, score=0.294858 -   2.9s
[CV] alpha=1.0, fit_prior=True .......................................
[CV] .............. alpha=1.0, fit_prior=True, score=0.295935 -   2.9s
[CV] alpha=1.0, fit_prior=False ......................................
[CV] ............. alpha=1.0, fit_prior=False, score=0.297254 -   3.0s
[CV] alpha=1.0, fit_prior=False ......................................
[CV] ............. alpha=1.0, fit_prior=False, score=0.294880 -   2.9s
[CV] alpha=1.0, fit_prior=False ......................................
[CV] ............. alpha=1.0, fit_prior=False, score=0.295969 -   3.0s
[CV] alpha=0.1, fit_prior=True .......................................
[CV] .............. alpha=0.1, fit_prior=True, score=0.297282 -   3.0s
[CV] alpha=0.1, fit_prior=True .......................................
[CV] .

[Parallel(n_jobs=10)]: Done  36 out of  36 | elapsed:  2.2min finished


Random Search took 138.141034126 seconds to run
Grid Search took 0.000313997268677 seconds to run
             precision    recall  f1-score   support

          0       0.71      0.72      0.71     47103
          1       0.29      0.27      0.28     19520

avg / total       0.58      0.59      0.59     66623

             precision    recall  f1-score   support

          0       0.71      0.72      0.71     93945
          1       0.30      0.28      0.29     39301

avg / total       0.59      0.59      0.59    133246

             precision    recall  f1-score   support

          0       0.71      0.72      0.71    141314
          1       0.29      0.28      0.28     58555

avg / total       0.59      0.59      0.59    199869

             precision    recall  f1-score   support

          0       0.71      1.00      0.83    188401
          1       0.30      0.00      0.01     78091

avg / total       0.59      0.71      0.59    266492

             precision    recall  f1-score

In [78]:
coln = 1
print(cv_result.groupby(cv_result.columns[coln])['mean_score'].mean())
print('\n std_deviation is')
print(cv_result.groupby(cv_result.columns[coln])['mean_score'].std())

alpha
0.00     0.500891
0.01     0.500891
0.10     0.500891
0.50     0.500891
1.00     0.500891
10.00    0.500911
Name: mean_score, dtype: float64

 std_deviation is
alpha
0.00     0.000066
0.01     0.000066
0.10     0.000066
0.50     0.000066
1.00     0.000066
10.00    0.000088
Name: mean_score, dtype: float64


# Results
* fitting prior has better mean and almost same std
* alpha should be only > 0. But difference between positive alphas is negligible, even on different exponential orders
* 

## Logistic Regression cross validation

In [54]:

clf =  LogisticRegressionCV(n_jobs =3, class_weight= 'balanced', scoring = 'f1',cv=3)

clf.fit(X_train,y_train)


predicted_proba = pd.DataFrame()
predicted_proba['prob'] = clf.predict_proba(X_test).max(axis=1)
predicted_proba['class'] = clf.predict(X_test)
predicted_proba[gt] =  y_test
predicted_proba.sort_values(by = 'prob',ascending = False, inplace = True)

cut_percentages = [10,20,30,40]
#converted = (predicted_proba['class']==1)
#me fijo todos los users que van a convertir en el futuro


start = 0
mobility_dict = {}
index_segments = []
for i in cut_percentages:
    cut_size = int(len(predicted_proba)*i/100.0)    
    bool_array = (~np.ones(len(predicted_proba), dtype=bool))
    bool_array[start: start + cut_size] = True
    index_segments += [bool_array]
    start = start + cut_size
#    subtable = 
    #mobility_dict[str(i)] = predicted_proba.loc[index_segments[-1],'ref_hash']
    
    print(classification_report(predicted_proba.loc[index_segments[-1],gt].values,
                            predicted_proba.loc[index_segments[-1],'class'].values ))
    
    
print(classification_report(predicted_proba.loc[:,gt].values,
                            predicted_proba.loc[:,'class'].values ))
    
    
cv_result = pd.DataFrame(columns=['mean_score' ] + [key for key, value in clf.grid_scores_[0].parameters.items() ] )
i=0
for setup in clf.grid_scores_:
    row = [clf.grid_scores_[i].mean_validation_score ] + [value for key, value in clf.grid_scores_[i].parameters.items() ]
    cv_result.loc[i] = row
    i+=1
#este flag es basicamente para chequear que hayamos terminado nomas
finished = True

all_time =   time.time() - start_time

#este flag es basicamente para chequear que hayamos terminado nomas
finished = True

print('This cell took %s seconds to run' % all_time)


             precision    recall  f1-score   support

          0       0.67      0.85      0.75     43545
          1       0.40      0.19      0.25     22464

avg / total       0.58      0.63      0.58     66009

             precision    recall  f1-score   support

          0       0.64      0.75      0.69     83353
          1       0.40      0.28      0.33     48666

avg / total       0.55      0.58      0.56    132019

             precision    recall  f1-score   support

          0       0.63      0.67      0.65    124185
          1       0.39      0.35      0.37     73843

avg / total       0.54      0.55      0.55    198028

             precision    recall  f1-score   support

          0       0.62      0.56      0.59    164393
          1       0.38      0.44      0.41     99645

avg / total       0.53      0.52      0.52    264038

             precision    recall  f1-score   support

          0       0.64      0.66      0.65    415477
          1       0.39      0.36 

AttributeError: 'LogisticRegressionCV' object has no attribute 'grid_scores_'

## Cross Validate SGD

In [ ]:
param_grid = [
  { 'alpha': [1e-3,5*1e-3,1e-2], 'l1_ratio': [3*1e-2,1e-3,6*1e-4], 'n_iter': [2,3]
  }
    ]

In [37]:
start_time = time.time()

param_grid = [
  { 'alpha': [1e-3,5*1e-3,1e-2], 'l1_ratio': [3*1e-2,1e-3,6*1e-4], 'n_iter': [2,3]
  }
    ]
sgd = SGDClassifier(loss='modified_huber', penalty='elasticnet', 
             fit_intercept=True,  shuffle=True, 
                    n_jobs=3,learning_rate='optimal', power_t =2, eta0 =5,
                    class_weight='balanced', average=40)

clf =GridSearchCV(sgd, param_grid, scoring='roc_auc', fit_params=None, n_jobs=4, iid=True, refit=True, 
verbose=3, pre_dispatch='2*n_jobs', error_score='raise')

clf.fit(X_train,y_train)



predicted_proba = pd.DataFrame()
predicted_proba['prob'] = clf.predict_proba(X_test).max(axis=1)
predicted_proba['class'] = clf.predict(X_test)
predicted_proba[gt] =  y_test
predicted_proba.sort_values(by = 'prob',ascending = False, inplace = True)

cut_percentages = [10,20,30,40]
#converted = (predicted_proba['class']==1)
#me fijo todos los users que van a convertir en el futuro


start = 0
mobility_dict = {}
index_segments = []
for i in cut_percentages:
    cut_size = int(len(predicted_proba)*i/100.0)    
    bool_array = (~np.ones(len(predicted_proba), dtype=bool))
    bool_array[start: start + cut_size] = True
    index_segments += [bool_array]
    start = start + cut_size
#    subtable = 
    #mobility_dict[str(i)] = predicted_proba.loc[index_segments[-1],'ref_hash']
    
    print(classification_report(predicted_proba.loc[index_segments[-1],gt].values,
                            predicted_proba.loc[index_segments[-1],'class'].values ))
    
    
print(classification_report(predicted_proba.loc[:,gt].values,
                            predicted_proba.loc[:,'class'].values ))
    
    
cv_result = pd.DataFrame(columns=['mean_score' ] + [key for key, value in clf.grid_scores_[0].parameters.items() ] )
i=0
for setup in clf.grid_scores_:
    row = [clf.grid_scores_[i].mean_validation_score ] + [value for key, value in clf.grid_scores_[i].parameters.items() ]
    cv_result.loc[i] = row
    i+=1
#este flag es basicamente para chequear que hayamos terminado nomas
finished = True

all_time =   time.time() - start_time

#este flag es basicamente para chequear que hayamos terminado nomas
finished = True

print('This cell took %s seconds to run' % all_time)


Fitting 3 folds for each of 18 candidates, totalling 54 fits


/home/ubuntu/Virtualenvs/mateo/lib/python3.4/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)


[CV] n_iter=2, alpha=0.001, l1_ratio=0.03 ............................


/home/ubuntu/Virtualenvs/mateo/lib/python3.4/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)


[CV] n_iter=2, alpha=0.001, l1_ratio=0.03 ............................


...

[CV]  n_iter=2, alpha=0.01, l1_ratio=0.0006000000000000001, score=0.122741 -   3.5s
[CV] n_iter=3, alpha=0.01, l1_ratio=0.0006000000000000001 ............


/home/ubuntu/Virtualenvs/mateo/lib/python3.4/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)


[CV]  n_iter=3, alpha=0.01, l1_ratio=0.0006000000000000001, score=0.132433 -   3.7s
[CV] n_iter=3, alpha=0.01, l1_ratio=0.0006000000000000001 ............
[CV]  n_iter=3, alpha=0.01, l1_ratio=0.0006000000000000001, score=0.135111 -   3.4s
[CV]  n_iter=3, alpha=0.01, l1_ratio=0.0006000000000000001, score=0.135908 -   3.1s


[Parallel(n_jobs=4)]: Done  54 out of  54 | elapsed:  1.7min finished


             precision    recall  f1-score   support

          0       1.00      0.31      0.47    355992
          1       0.06      0.99      0.11     15152

avg / total       0.96      0.33      0.45    371144

This cell took 108.56432461738586 seconds to run


In [ ]:
threshold = 0.7
ind = predicted_proba['prob']>threshold
print(classification_report(predicted_proba.loc[ind,'y'].values,predicted_proba.loc[ind,'class'].values ))

In [38]:
clf.best_estimator_

SGDClassifier(alpha=0.005, average=40, class_weight='balanced', epsilon=0.1,
       eta0=5, fit_intercept=True, l1_ratio=0.0006000000000000001,
       learning_rate='optimal', loss='modified_huber', n_iter=3, n_jobs=3,
       penalty='elasticnet', power_t=2, random_state=None, shuffle=True,
       verbose=0, warm_start=False)

In [39]:
coln=1
print(cv_result.groupby(cv_result.columns[coln])['mean_score'].mean())
print(cv_result.groupby(cv_result.columns[coln])['mean_score'].std())

n_iter
2.0    0.125741
3.0    0.132943
Name: mean_score, dtype: float64
n_iter
2.0    0.002609
3.0    0.002703
Name: mean_score, dtype: float64


### Results
* l1_ratio = cuanto mas chico mejor con lo cual la perdida l2 parece ser mejor
* alpha = 1e-3 es suficiente pues casi no afecta el score
* power_t = muy variado, no parece haber correlacion entre el tamanyo y el avg, mean_score
* eta0 = no afecta mucho pero parece ser que con ser >1 ya esta
* power_t = parecen ser practicamente iguales
* n_iter = cuanto mas iter NO mejora el score

In [51]:
len(predicted_proba)

371161

## Cross Validate NN

In [35]:
start_time = time.time()

X_train,X_test,y_train ,y_test = train_test_split(X,y, test_size=0.2, random_state=42)

param_grid = { 'alpha': [1e-1,5*1e-2,1e-2],
              'hidden_layer_sizes':[(50,),(15,5)],
              'learning_rate': ['adaptive',"invscaling"],
              "algorithm": ['adam'],'momentum': [1e-2, 1e-1, 0.5],
  'power_t': [1e-3, 5*1e-4, 1e-5], 'activation':['logistic','relu']
 }

mlp = MLPClassifier(shuffle=True, 
                 verbose=True)

clf =GridSearchCV(mlp, param_grid, scoring='roc_auc', fit_params=None, n_jobs=4, iid=True, refit=True, 
verbose=3, pre_dispatch='2*n_jobs', error_score='raise')

clf.fit(X_train,y_train)

predicted_proba = pd.DataFrame()
predicted_proba['prob'] = clf.predict_proba(X_test).max(axis=1)
predicted_proba['gt'] = y_test
predicted_proba['class'] = clf.predict(X_test)

threshold = 0.7
ind = predicted_proba['prob']>threshold

all_time =   time.time() - start_time

print(classification_report(predicted_proba.loc[ind,'gt'].values,predicted_proba.loc[ind,'class'].values ))

cv_result = pd.DataFrame(columns=['mean_score' ] + [key for key, value in clf.grid_scores_[0].parameters.items() ] )
i=0
for setup in clf.grid_scores_:
    row = [clf.grid_scores_[i].mean_validation_score ] + [value for key, value in clf.grid_scores_[i].parameters.items() ]
    cv_result.loc[i] = row
    i+=1
#este flag es basicamente para chequear que hayamos terminado nomas
finished = True

Fitting 3 folds for each of 144 candidates, totalling 432 fits
[CV] algorithm=adam, hidden_layer_sizes=(50,), activation=logistic, power_t=0.001, alpha=0.1, learning_rate=adaptive 
[CV] algorithm=adam, hidden_layer_sizes=(50,), activation=logistic, power_t=0.001, alpha=0.1, learning_rate=adaptive 
[CV] algorithm=adam, hidden_layer_sizes=(50,), activation=logistic, power_t=0.001, alpha=0.1, learning_rate=adaptive 
[CV] algorithm=adam, hidden_layer_sizes=(50,), activation=logistic, power_t=0.0005, alpha=0.1, learning_rate=adaptive 
Iteration 1, loss = 0.20405210
Iteration 1, loss = 0.20380843
Iteration 1, loss = 0.20378706
Iteration 1, loss = 0.20405210


/home/teo/Repositories/scikit-learn/sklearn/metrics/classification.py:1097: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


[CV]  algorithm=adam, hidden_layer_sizes=(50,), activation=logistic, power_t=0.0005, alpha=0.1, learning_rate=adaptive, score=0.000000 -   1.5s
[CV] algorithm=adam, hidden_layer_sizes=(50,), activation=logistic, power_t=0.0005, alpha=0.1, learning_rate=adaptive 


/home/teo/Repositories/scikit-learn/sklearn/metrics/classification.py:1097: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


[CV]  algorithm=adam, hidden_layer_sizes=(50,), activation=logistic, power_t=0.001, alpha=0.1, learning_rate=adaptive, score=0.000000 -   1.7s


/home/teo/Repositories/scikit-learn/sklearn/metrics/classification.py:1097: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


[CV]  algorithm=adam, hidden_layer_sizes=(50,), activation=logistic, power_t=0.001, alpha=0.1, learning_rate=adaptive, score=0.000000 -   2.1s


/home/teo/Repositories/scikit-learn/sklearn/metrics/classification.py:1097: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


[CV]  algorithm=adam, hidden_layer_sizes=(50,), activation=logistic, power_t=0.001, alpha=0.1, learning_rate=adaptive, score=0.000000 -   1.9s


KeyboardInterrupt: 

In [1]:
clf.best_estimator_

NameError: name 'clf' is not defined

In [52]:
# compare best parameters to tune
coln=1
cv_result.groupby(cv_result.columns[coln])['mean_score'].mean()
cv_result.groupby(cv_result.columns[coln])['mean_score'].std()

alpha
0.00     0.000007
0.01     0.000007
0.10     0.000007
0.50     0.000007
1.00     0.000007
10.00    0.000007
Name: mean_score, dtype: float64

### Results
* activation = logistic es 10% mejor
* alpha = 1e-2 el mejor 
* power_t = cuanto mas chico mejor, 1e-3 por lo menos
* hidden_layer_size = menos layers es mejor..?
* power_t = parecen ser practicamente iguales
* n_iter = cuanto mas iter NO mejora el score

## Bernoulli RBM features selection & Logit crossval

In [127]:
start_time = time.time()

## obs. este metodo es especial y asume que todos los valores son True/False o que \in [0,1]
# luego tengo que pensar en como tomar los features nuevamente.
# tampoco 'fittea' en la forma tradicional. Sino que se le puede tomar al y como un feature mas y esta red
# va 'modificando' todos los valores del X (minimizando la entropia) para dar un output. Luego corriendo 
# clf.gibbs(X_test) con el y_test como feature tmb, nos transforma la data para ver el output como la 'prediccion'
# obviamente no tiene probabilidades


#X = X_train[X_cols].values
#y = X_train['ground_truth'].values

df = X_train.drop(X_train[X_train[X_train.columns[0:3]].\
                                   sum(axis=1)==0].index)
df = df[X_cols + ['ground_truth']]

for col in X_train.columns[0:3]:
    df[col] = df[col]*1.0/df[df.columns[0:3]].sum(axis=1)

df[df.columns[3]] =  df[df.columns[3]]/df[df.columns[3]].max()
df[df.columns[4]] =  df[df.columns[4]]/df[df.columns[4]].max()

X = df[df.columns[:-1]].values
y = df['ground_truth'].values


param_grid = {'rbm__n_components': [256, 128,46,10],
   'rbm__n_iter':[15,10,5], 'rbm__learning_rate': [1e-4,1e-3,1e-2,1e-1,5*1e-3,5*1e-2,5*1e-1],
  'rbm__batch_size': [10e4,3*10e3, 1e3, 300],"logistic__C": [1.0, 10.0, 100.0] 
 }

rbm = BernoulliRBM(verbose=True)
logistic = LogisticRegression()
classifier = Pipeline([("rbm", rbm), ("logistic", logistic)])

X_train,X_test,y_train ,y_test = train_test_split(X,y, test_size=0.2, random_state=42)

clf =GridSearchCV(classifier, param_grid, scoring='roc_auc', fit_params=None, n_jobs=4, iid=True, refit=True, 
verbose=3, pre_dispatch='2*n_jobs', error_score='raise')

clf.fit(X_train,y_train)

#este metodo NO tiene predicted proba, lo que hacemos es recorrer 
#predicted_labels = rbm.gibbs(X_test)[:,-1]
#real_labels = X_test[:,-1]
#print(classification_report(real_labels,predicted_labels ))


predicted_proba = pd.DataFrame()
predicted_proba['prob'] = clf.predict_proba(X_test).max(axis=1)
predicted_proba['gt'] = y_test
predicted_proba['class'] = clf.predict(X_test)

threshold = 0.7
ind = predicted_proba['prob']>threshold
print(classification_report(predicted_proba.loc[ind,'gt'].values,predicted_proba.loc[ind,'class'].values ))
all_time =   time.time() - start_time

finished = True

cv_result = pd.DataFrame(columns=['mean_score' ] + [key for key, value in clf.grid_scores_[0].parameters.items() ] )
i=0
for setup in clf.grid_scores_:
    row = [clf.grid_scores_[i].mean_validation_score ] + [value for key, value in clf.grid_scores_[i].parameters.items() ]
    cv_result.loc[i] = row
    i+=1
    
print('This cell took %s seconds to run' % all_time)


KeyboardInterrupt: 

In [ ]:
coln=1
print(cv_result.groupby(cv_result.columns[coln])['mean_score'].mean())
print(cv_result.groupby(cv_result.columns[coln])['mean_score'].std())

## ADaboost CV

In [41]:
rf_param_grid = {'n_estimators': [10, 30, 100, 300, 1000]}
boost_param_grid = {'n_estimators': [10, 30, 100, 300, 1000],
                      'max_depth': [2, 3, 4, 5],
                      'min_samples_leaf': [1, 2, 3]}
ada_param_grid = {'n_estimators': [10, 30, 100, 300, 1000],
                   'learning_rate': [0.1, 0.3, 1.0, 3.0]}

X_train,X_test,y_train ,y_test = train_test_split(X,y, test_size=0.2, random_state=42)
#rf_est = RandomForestClassifier()
#rf_gs_cv = GridSearchCV(rf_est, rf_param_grid).fit(X_train, Y_train)
#print(rf_gs_cv.best_score_, rf_gs_cv.best_params_)
print('\n')
 
#boost_est = GradientBoostingClassifier()
#boost_gs_cv = GridSearchCV(boost_est, boost_param_grid).fit(X_train, y_train)
#print(boost_gs_cv.best_score_, boost_gs_cv.best_params_)
print('\n')
 
ada_est = AdaBoostClassifier()
ada_gs_cv = GridSearchCV(ada_est, ada_param_grid).fit(X_train, y_train)
print(ada_gs_cv.best_score_, ada_gs_cv.best_params_)
print('\n')

KeyboardInterrupt: 

In [42]:
start_time = time.time()


param_grid = {'n_estimators': [200,600,60,45,30,15],
 'learning_rate': [1e-4,1e-3,1e-2,1e-1,5*1e-3,5*1e-2,5*1e-1]
 }

aboost = AdaBoostClassifier(algorithm='SAMME.R')

clf =GridSearchCV(aboost, param_grid, scoring='roc_auc', fit_params=None, n_jobs=4, iid=True, refit=True, 
verbose=3, pre_dispatch='2*n_jobs', error_score='raise')

clf.fit(X_train,y_train)

predicted_proba = pd.DataFrame()
predicted_proba['prob'] = clf.predict_proba(X_test).max(axis=1)
predicted_proba['gt'] = y_test
predicted_proba['class'] = clf.predict(X_test)

threshold = 0.7
ind = predicted_proba['prob']>threshold
print(classification_report(predicted_proba.loc[ind,'gt'].values,predicted_proba.loc[ind,'class'].values ))
all_time =   time.time() - start_time

print('This cell took %s seconds to run' % all_time)

finished = True
#este flag es basicamente para chequear 

cv_result = pd.DataFrame(columns=['mean_score' ] + [key for key, value in clf.grid_scores_[0].parameters.items() ] )
i=0
for setup in clf.grid_scores_:
    row = [clf.grid_scores_[i].mean_validation_score ] + [value for key, value in clf.grid_scores_[i].parameters.items() ]
    cv_result.loc[i] = row
    i+=1

finished = True

Fitting 3 folds for each of 42 candidates, totalling 126 fits
[CV] n_estimators=200, learning_rate=0.0001 ..........................
[CV] n_estimators=200, learning_rate=0.0001 ..........................
[CV] n_estimators=200, learning_rate=0.0001 ..........................
[CV] n_estimators=600, learning_rate=0.0001 ..........................
[CV] n_estimators=600, learning_rate=0.0001 ..........................


KeyboardInterrupt: 

In [23]:
coln=3
print(cv_result.groupby(cv_result.columns[coln])['mean_score'].mean())
print(cv_result.groupby(cv_result.columns[coln])['mean_score'].std())


criterion
gini    0.802919
Name: mean_score, dtype: float64
criterion
gini    0.000598
Name: mean_score, dtype: float64


## CrossV SVC

In [ ]:
start_time = time.time()

param_grid = [{'kernel': ['rbf'], 'gamma': list(10.0 ** np.arange(-2, 3)),
                     'C': list(10.0 ** np.arange(0, 4))},
                    {'kernel': ['poly'], 'C': list(10.0 ** np.arange(0, 4)), 'degree'[2,3,4]}]

svc = SVC(shuffle=True, probability=True,decision_function_shape = 'ovr',
           verbose=True, class_weight='balanced'
          )

clf =GridSearchCV(svc, param_grid, scoring='roc_auc', fit_params=None, n_jobs=4, iid=True, refit=True, 
verbose=3, pre_dispatch='2*n_jobs', error_score='raise')

clf.fit(X_train,y_train)

predicted_proba = pd.DataFrame()
predicted_proba['prob'] = clf.predict_proba(X_test).max(axis=1)
predicted_proba['gt'] = y_test
predicted_proba['class'] = clf.predict(X_test)

threshold = 0.7
ind = predicted_proba['prob']>threshold

all_time =   time.time() - start_time

print(classification_report(predicted_proba.loc[ind,'gt'].values,predicted_proba.loc[ind,'class'].values ))

cv_result = pd.DataFrame(columns=['mean_score' ] + [key for key, value in clf.grid_scores_[0].parameters.items() ] )
i=0
for setup in clf.grid_scores_:
    row = [clf.grid_scores_[i].mean_validation_score ] + [value for key, value in clf.grid_scores_[i].parameters.items() ]
    cv_result.loc[i] = row
    i+=1
#este flag es basicamente para chequear que hayamos terminado nomas
finished = True

In [ ]:
coln=3
print(cv_result.groupby(cv_result.columns[coln])['mean_score'].mean())
print(cv_result.groupby(cv_result.columns[coln])['mean_score'].std())


In [214]:
#tomo tiempo para saber cuanto tarda fitear el estimador
start_time = time.time()

#model = model.fit(X,y,sample_weight=W)

W = np.array([10 if i == 1 or i ==2  else 1 for i in y_mini])
gradboost.fit(X_mini,y_mini, sample_weight=W)

elapsed_time =   time.time() - start_time
print('GradientBoosting took %s seconds to run' % elapsed_time)

#validated =  cross_val_score(gradboost,X,y,cv=5, scoring = "f1_weighted")


all_time =   time.time() - start_time
print('This cell took %s seconds to run' % all_time)

GradientBoosting took 117.26272392272949 seconds to run
This cell took 117.26284193992615 seconds to run


## Timings
* 5s con  10 n_estimadores, 15 max_depth y X.sample(0.2)
* 15s con 10 n_estimadores, 15 max_depth y X.sample(0.2)
* 47s con 10 n_estimadores, 15 max_depth y X.sample(0.4)
* 35s con 10 n_estimadores, 15 max_depth y X.sample(0.4)
* 117 con 20 n_estimadores, 20 max_depth y X.sample(0.4)

In [31]:
X_train['ground_truth'].value_counts()

0    128153
1      7064
Name: ground_truth, dtype: int64

## Cross Validate - Random Forest

In [ ]:
param_grid = {'criterion': ['gini'], 'n_estimators': [15,30,50],
  'max_features': ["auto", "log2"], "bootstrap": [ False, True],
    "min_samples_leaf": np.append(np.random.randint(3,15,3),[3]),'max_depth':[10,20,30], 
               "class_weight": ['balanced']
              }

In [86]:
start_time = time.time()

param_grid = {'criterion': ['gini'], 'n_estimators': [25],
  'max_features': ["auto"], "bootstrap": [ False],
    "min_samples_leaf":[6,12],'max_depth':[15,25], 
               "class_weight": ['balanced']
              }
rforest  = RandomForestClassifier( )

clf =GridSearchCV(rforest, param_grid, scoring='roc_auc', fit_params=None, n_jobs=8, iid=True, refit=True, 
verbose=3, pre_dispatch='2*n_jobs', error_score='raise')

clf2 = RandomForestClassifier()

X = X_train[X_train.columns[:-1]].values
Y = X_train['y'].values

#X = X_train_categorical
clf.fit(X,Y)


#random_search.fit(X_train,y_train)
elapsed_time =  time.time() - start_time

print('Random Search took %s seconds to run' % elapsed_time)


all_time =   time.time() - start_time 
print('Grid Search took %s seconds to run' % (all_time - elapsed_time))


X_test = X_val[X_train.columns[:-1]].values
Y_test = X_val['y'].values

predicted_proba = pd.DataFrame()
predicted_proba['prob'] = clf.predict_proba(X_test).max(axis=1)
predicted_proba.index = X_val.index
predicted_proba['class'] = clf.predict(X_test)
predicted_proba['gt'] =  Y_test
predicted_proba.index.name = "USER"
predicted_proba.sort_values(by = 'prob',ascending = False, inplace = True)

cut_percentages = [10,20,30,40]
#converted = (predicted_proba['class']==1)
#me fijo todos los users que van a convertir en el futuro

past_epidemic = X_val[X_val['y']==1].index.values

start = 0
mobility_dict = {}
index_segments = []
for i in cut_percentages:
    cut_size = int(len(predicted_proba)*i/100.0)    
    bool_array = (~np.ones(len(predicted_proba), dtype=bool))
    bool_array[start: start + cut_size] = True
    index_segments += [bool_array]
    start = start + cut_size
    #subtable = 
    mobility_dict[str(i)] = predicted_proba.loc[index_segments[-1]].index.values
    
    print(classification_report(predicted_proba.loc[index_segments[-1],"gt"].values,
                            predicted_proba.loc[index_segments[-1],'class'].values ))
    
    
print(classification_report(predicted_proba.loc[:,"gt"].values,
                            predicted_proba.loc[:,'class'].values ))
    
    
print("* Rates for distinct percentage groups \n")
for i in cut_percentages:
    subtable = mobility_dict[str(i)]

    print("{0} %% group rate: {1} size: {2}".format(str(i), 
                                np.in1d(subtable,past_epidemic).sum()*1.0/len(subtable),len(subtable))) 
    

clf2.set_params(n_jobs = 6,**clf.best_params_)
clf2.fit(X,Y)
    
cv_result = pd.DataFrame(columns=['mean_score' ] + [key for key, value in clf.grid_scores_[0].parameters.items() ] )
i=0
for setup in clf.grid_scores_:
    row = [clf.grid_scores_[i].mean_validation_score ] + [value for key, value in clf.grid_scores_[i].parameters.items() ]
    cv_result.loc[i] = row
    i+=1

    
    
all_time =   time.time() - start_time

#este flag es basicamente para chequear que hayamos terminado nomas
finished = True

print('This cell took %s seconds to run' % all_time)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] bootstrap=False, min_samples_leaf=6, n_estimators=25, criterion=gini, max_features=auto, max_depth=15, class_weight=balanced 
[CV] bootstrap=False, min_samples_leaf=6, n_estimators=25, criterion=gini, max_features=auto, max_depth=15, class_weight=balanced 
[CV] bootstrap=False, min_samples_leaf=6, n_estimators=25, criterion=gini, max_features=auto, max_depth=15, class_weight=balanced 
[CV] bootstrap=False, min_samples_leaf=12, n_estimators=25, criterion=gini, max_features=auto, max_depth=15, class_weight=balanced 
[CV] bootstrap=False, min_samples_leaf=12, n_estimators=25, criterion=gini, max_features=auto, max_depth=15, class_weight=balanced 
[CV] bootstrap=False, min_samples_leaf=12, n_estimators=25, criterion=gini, max_features=auto, max_depth=15, class_weight=balanced 
[CV] bootstrap=False, min_samples_leaf=6, n_estimators=25, criterion=gini, max_features=auto, max_depth=25, class_weight=balanced 
[CV] bootstrap=False

[Parallel(n_jobs=8)]: Done  13 out of  12 | elapsed:  4.2min remaining:  -19.6s


[CV] bootstrap=False, min_samples_leaf=12, n_estimators=25, criterion=gini, max_features=auto, max_depth=25, class_weight=balanced 
[CV]  bootstrap=False, min_samples_leaf=6, n_estimators=25, criterion=gini, max_features=auto, max_depth=15, class_weight=balanced, score=0.500056 - 4.2min


[Parallel(n_jobs=8)]: Done  13 out of  12 | elapsed:  4.3min remaining:  -19.9s


[CV] bootstrap=False, min_samples_leaf=12, n_estimators=25, criterion=gini, max_features=auto, max_depth=25, class_weight=balanced 
[CV]  bootstrap=False, min_samples_leaf=12, n_estimators=25, criterion=gini, max_features=auto, max_depth=15, class_weight=balanced, score=0.498584 - 4.2min


...

[CV]  bootstrap=False, min_samples_leaf=12, n_estimators=25, criterion=gini, max_features=auto, max_depth=25, class_weight=balanced, score=0.500189 - 5.3min


[Parallel(n_jobs=8)]: Done  13 out of  12 | elapsed:  9.7min remaining:  -44.7s


[CV]  bootstrap=False, min_samples_leaf=12, n_estimators=25, criterion=gini, max_features=auto, max_depth=25, class_weight=balanced, score=0.499681 - 5.3min


[Parallel(n_jobs=8)]: Done  13 out of  12 | elapsed:  9.8min remaining:  -45.1s
[Parallel(n_jobs=8)]: Done  12 out of  12 | elapsed:  9.8min finished


Random Search took 1023.46121788 seconds to run
Grid Search took 0.000302076339722 seconds to run
             precision    recall  f1-score   support

          0       0.71      0.92      0.80     47125
          1       0.29      0.08      0.13     19498

avg / total       0.58      0.67      0.60     66623

             precision    recall  f1-score   support

          0       0.71      0.77      0.74     94223
          1       0.29      0.23      0.26     39023

avg / total       0.59      0.61      0.60    133246

             precision    recall  f1-score   support

          0       0.71      0.66      0.68    141157
          1       0.30      0.35      0.32     58712

avg / total       0.59      0.57      0.57    199869

             precision    recall  f1-score   support

          0       0.71      0.57      0.63    188258
          1       0.29      0.43      0.35     78234

avg / total       0.59      0.53      0.55    266492

             precision    recall  f1-score

In [85]:
np.bincount(Y)[0]*1.0/np.bincount(Y).sum(), np.bincount(Y)[1]*1.0/np.bincount(Y).sum()

(0.70631558433771058, 0.29368441566228937)

In [90]:
clf2 = RandomForestClassifier()
clf2.set_params(n_jobs = 6,**clf.best_params_)
clf2.fit(X,Y)

RandomForestClassifier(bootstrap=False, class_weight='balanced',
            criterion='gini', max_depth=25, max_features='auto',
            max_leaf_nodes=None, min_samples_leaf=12, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=25, n_jobs=6,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [92]:
#OJO esta parte con poly = True no funciona.
feature_importance = pd.DataFrame(clf2.feature_importances_,columns=['prob'])
feature_importance = feature_importance.sort_values(by = 'prob',ascending=False)
selected_features = X_train.columns[feature_importance.index]

#guardo los labels como estan en las tablas (con numeros y no palabras)
feature_importance['labels'] = selected_features.values

In [93]:
feature_importance

,prob,labels
2,0.047771,ANTENNA_ID_1
0,0.047296,ANTENNA_ID_0
6,0.046120,ANTENNA_ID_3
4,0.045383,ANTENNA_ID_2
20,0.045289,MOBILITY_DIAMETER
8,0.043635,ANTENNA_ID_4
10,0.040790,ANTENNA_ID_5
25,0.040621,TimeWeekDaylight
12,0.040533,ANTENNA_ID_6
14,0.038278,ANTENNA_ID_7


In [43]:
X_train.shape,y_train.shape

((1600644, 38), (1600644,))

In [35]:
X_train.shape

(1600644, 38)

In [51]:
clf.best_estimator_, clf.best_score_

(RandomForestClassifier(bootstrap=False, class_weight='balanced',
             criterion='gini', max_depth=15, max_features='auto',
             max_leaf_nodes=None, min_samples_leaf=4, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=35, n_jobs=1,
             oob_score=False, random_state=None, verbose=0,
             warm_start=False), 0.54557839409818476)

In [17]:
a = classification_report(predicted_proba.loc[ind,'y'].values,predicted_proba.loc[ind,'class'].values )

In [60]:
coln = 3
print(cv_result.groupby(cv_result.columns[coln])['mean_score'].mean())
print(cv_result.groupby(cv_result.columns[coln])['mean_score'].std())

n_estimators
20.0    0.537925
35.0    0.541880
Name: mean_score, dtype: float64
n_estimators
20.0    0.00539
35.0    0.00419
Name: mean_score, dtype: float64


### Results
* NO escalar (normalizar, restar la media  dividir por la std, etc) los resultados pues REVIENTAN los scores.
* bootstrap  = False es 5% mejor
* min_samples_leaf = mas chico es claramente mejor, pero tmb aumenta el overfitting lo cual me hace caer mucho el valor del recall en el test_set. Sin embargo es un parametro muy sensible en la precision. Resta evaluar asi el tradeoff entre la precision y el volumen de users al cual queremos llegar.
* n_estimators = aumentar mas de 30 no tendria mucho sentido
* citerion = entropy o gini no cambia. gini podria ser mejor entonces pues entropy usa logs de los valores lo cual es mas computacionalmente costoso
* max_features = no afecta al score. con auto esta bien
* max_depth =  mas es mejor. intentaria probar con >15

## Cross Validate ExtraTreeClassifier

In [94]:
predicted_proba = pd.DataFrame()
predicted_proba['prob'] = clf.predict_proba(X_train)[:,1]
predicted_proba['class'] = clf.predict(X_train)
predicted_proba['ref_hash'] =  train_table.index.values
predicted_proba['y'] =  y_train
predicted_proba.sort_values(by = 'prob',ascending = False, inplace = True)

cut_percentages = [10,20,30,40]
#converted = (predicted_proba['class']==1)
#me fijo todos los users que van a convertir en el futuro
past_epidemic = test_table[test_table['y']>0]['ref_hash'].values


start = 0
mobility_dict = {}
index_segments = []
for i in cut_percentages:
    cut_size = int(len(predicted_proba)*i/100.0)    
    bool_array = (~np.ones(len(predicted_proba), dtype=bool))
    bool_array[start: start + cut_size] = True
    index_segments += [bool_array]
    start = start + cut_size
#    subtable = 
    mobility_dict[str(i)] = predicted_proba.loc[index_segments[-1],'ref_hash']
    
    print(classification_report(predicted_proba.loc[index_segments[-1],'y'].values,
                            predicted_proba.loc[index_segments[-1],'class'].values ))
    
print("Rates for %s app %s " % (appid_name[application_id], application_id))   
for i in cut_percentages:
    subtable = mobility_dict[str(i)]
    
    print("{0} %% group rate: {1} size: {2}".format(str(i), subtable.isin(past_epidemic).sum()*1.0/len(subtable), 
                                       len(subtable))) 


/home/ubuntu/Virtualenvs/mateo/lib/python3.4/site-packages/sklearn/metrics/classification.py:1117: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


             precision    recall  f1-score   support

          0       0.00      0.00      0.00     26720
          1       0.25      1.00      0.40      8738

avg / total       0.06      0.25      0.10     35458

             precision    recall  f1-score   support

          0       0.97      0.43      0.60     66920
          1       0.07      0.76      0.14      3996

avg / total       0.92      0.45      0.57     70916

             precision    recall  f1-score   support

          0       0.99      1.00      0.99    105255
          1       0.00      0.00      0.00      1119

avg / total       0.98      0.99      0.98    106374

             precision    recall  f1-score   support

          0       0.98      1.00      0.99    139582
          1       0.00      0.00      0.00      2250

avg / total       0.97      0.98      0.98    141832

Rates for dafiti app 189 
10 %% group rate: 0.10429240227875233 size: 35458
20 %% group rate: 0.024479666083817474 size: 70916
30 %% group r

In [100]:
coln = 2
print(cv_result.groupby(cv_result.columns[coln])['mean_score'].mean())
print(cv_result.groupby(cv_result.columns[coln])['mean_score'].std())

alpha
0.00     0.000000
0.01     0.242260
0.10     0.242266
0.50     0.242220
1.00     0.242200
10.00    0.241269
Name: mean_score, dtype: float64
alpha
0.00     0.000000
0.01     0.017568
0.10     0.017598
0.50     0.017578
1.00     0.017570
10.00    0.017780
Name: mean_score, dtype: float64


## Cross validate MNB

In [72]:
start_time = time.time()

# pero tenemos que transformar a y en un vector canonico indicando en 1 en la posicion correspondiente a su categoria
#y =  label_binarize(X_train['ground_truth'].values.astype(int),
#                    classes=list(set(X_train['ground_truth'].values.astype(int))))

perc = 0.3
mini_ind = np.random.choice(X_train.shape[0],int(perc*X_train.shape[0]),replace=False)
#X_mini =  X[mini_ind,:]
#y_mini = y[mini_ind]


clf = MultinomialNB(
        )

clf2 = MultinomialNB()

validated = []
# Only take the first fold.
n_folds = 4
for i in range(n_folds):
    # Break up the dataset into non-overlapping training (75%) and testing
    # (25%) sets.
    skf = StratifiedKFold(y_train, n_folds=n_folds)
    train_index, test_index = next(iter(skf))

    X_traincv = X_train[train_index]
    y_traincv = y_train[train_index]

    n_classes = len(np.unique(y_traincv))

    clf.fit(X_traincv,y_traincv)
    predictions = clf.predict(X_traincv)
        
    expected = y_traincv

    print(classification_report(expected, predictions))


predicted_proba = pd.DataFrame()
predicted_proba['prob'] = clf.predict_proba(X_test).max(axis=1)
predicted_proba['y'] = y_test
predicted_proba['class'] = clf.predict(X_test)

clf2.set_params(n_jobs = 6,**clf.best_params_)
clf2.fit(X,Y)


#este flag es basicamente para chequear que hayamos terminado nomas
finished = True

all_time =   time.time() - start_time
print('This cell took %s seconds to run' % all_time)


             precision    recall  f1-score   support

          0       0.99      0.85      0.91     70323
          1       0.12      0.73      0.20      1932

avg / total       0.97      0.84      0.89     72255

             precision    recall  f1-score   support

          0       0.99      0.85      0.91     70323
          1       0.12      0.73      0.20      1932

avg / total       0.97      0.84      0.89     72255

             precision    recall  f1-score   support

          0       0.99      0.85      0.91     70323
          1       0.12      0.73      0.20      1932

avg / total       0.97      0.84      0.89     72255

             precision    recall  f1-score   support

          0       0.99      0.85      0.91     70323
          1       0.12      0.73      0.20      1932

avg / total       0.97      0.84      0.89     72255

This cell took 1.06442093849 seconds to run


In [76]:
threshold = 0.99
ind = predicted_proba['prob']>threshold
print(classification_report(predicted_proba.loc[ind,'y'].values,predicted_proba.loc[ind,'class'].values ))


             precision    recall  f1-score   support

          0       0.99      0.87      0.93    185744
          1       0.15      0.73      0.26      5906

avg / total       0.96      0.87      0.91    191650



In [60]:
#tomo tiempo para saber cuanto tarda fitear el estimador
start_time = time.time()

svc2.fit(X_mini,y_mini)
elapsed_time =   time.time() - start_time
print('SVC took %s seconds to run' % elapsed_time)

all_time =   time.time() - start_time
print('This cell took %s seconds to run' % all_time)

SVC took 362.8217749595642 seconds to run
This cell took 362.8219118118286 seconds to run


## Timings
* 40k samples : 360s
* 

In [75]:
start_time =  time.time()
lr.fit(X_train,y_train)
elapsed_time =   time.time() - start_time
print('Logit took %s seconds to run' % elapsed_time)

Logit took 1073.0756621360779 seconds to run


In [ ]:
#tomo tiempo para saber cuanto tarda fitear el estimador
start_time = time.time()

lr.fit(X_train,y_train)
elapsed_time =   time.time() - start_time
print('Logit took %s seconds to run' % elapsed_time)

svc.fit(X_train,y_train)
elapsed_time =   time.time() - start_time
print('Linear SVC took %s seconds to run' % elapsed_time)

all_time =   time.time() - start_time
print('This cell took %s seconds to run' % all_time)

Logit took 89.15975689888 seconds to run
Linear SVC took 804.7178750038147 seconds to run

In [ ]:
y.value_counts

# Todo
* evaluate hit_rate and 
* tune adaboost, bernoulliRBM
* xgboost
* libffm
* SVC muy lento.. speed up in AWS
